In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# submission functionality
# ------------------------
import os, sys
sys.path.append(os.path.abspath("..")) # so src/ is on the path

import importlib
import submission_utils
importlib.reload(submission_utils) # force reload latest code

from submission_utils import save_submission
# ------------------------
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

In [13]:
# Load data
train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

target = "Milk_Yield_L"
id_col = "Cattle_ID"


def preprocess_pipeline(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()
    
    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )
    


            

    # Fill any remaining numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    # -----------------------
    # 3) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])
    
    

    # -----------------------
    # 5) Farm clustering
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        # Use only numeric features for clustering (exclude IDs)
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        # Aggregate per farm
        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()

        # Scale and cluster farms
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

        # Map back to rows
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    # -----------------------
    # 6) Drop raw IDs
    # -----------------------
    drop_cols = ["Cattle_ID", "Farm_ID"]
    milk_features = milk_features.drop(columns=[c for c in drop_cols if c in milk_features.columns])

    # -----------------------
    # 7) Optional one-hot encoding (we'll keep it OFF for CatBoost)
    # -----------------------
    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    # Make sure Farm_Cluster exists even if something went weird
    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    final_df = milk_features.copy()
    return final_df, y

In [ ]:
# # -----------------------------------
# # LightGBM hyperparameter search
# # -----------------------------------
# from sklearn.model_selection import RandomizedSearchCV

# lgb_base = lgb.LGBMRegressor(
#     objective="regression",
#     random_state=42
# )

# lgb_param_dist = {
#     "num_leaves": [31, 63, 127, 255],
#     "max_depth": [-1, 6, 8, 10],
#     "learning_rate": [0.01, 0.03, 0.05, 0.08],
#     "n_estimators": [300, 600, 900, 1200],
#     "subsample": [0.7, 0.8, 0.9],
#     "colsample_bytree": [0.7, 0.8, 0.9],
#     "min_child_samples": [10, 20, 50, 100]
# }

# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# lgb_search = RandomizedSearchCV(
#     estimator=lgb_base,
#     param_distributions=lgb_param_dist,
#     n_iter=25,                         # reduce if too slow
#     scoring="neg_mean_squared_error",  # we'll sqrt it to get RMSE
#     cv=kf,
#     verbose=1,
#     n_jobs=-1,
#     random_state=42,
# )

# print("Starting LightGBM hyperparameter search...")
# lgb_search.fit(X, y)

# best_lgb_params = lgb_search.best_params_
# best_lgb_rmse = np.sqrt(-lgb_search.best_score_)

# print("\nBest LightGBM params:")
# print(best_lgb_params)
# print(f"Best LightGBM CV RMSE: {best_lgb_rmse:.4f}")

In [64]:
# Build train and test matrices from the new pipeline
X, y = preprocess_pipeline(train, encode_flag=False, target_col=target)
test_df, _ = preprocess_pipeline(test, encode_flag=False, target_col=None)

print("Train shape:", X.shape)
print("Test shape:", test_df.shape)

Train shape: (209926, 41)
Test shape: (40000, 41)


In [65]:
# Categorical columns for CatBoost (from the processed X)
cat_features = X.select_dtypes(include="object").columns.tolist()
print("CatBoost categorical features:", cat_features)

CatBoost categorical features: ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type']


In [66]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import numpy as np

# 5-Fold Cross-Validation for CatBoost
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cat_fold_rmse = []
cat_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- CatBoost Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=1000,
        learning_rate=0.05,
        depth=6,          # you can try depth=6 like your teammate too
        subsample=0.8,
        random_seed=42,
        verbose=False
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    cat_fold_rmse.append(rmse)

    print(f"CatBoost Fold {fold+1} RMSE: {rmse:.4f}")
    cat_models.append(model)

print("\n==========================")
print(f"CatBoost Average RMSE: {np.mean(cat_fold_rmse):.4f}")
print("==========================\n")


----- CatBoost Fold 1 -----
CatBoost Fold 1 RMSE: 4.1080

----- CatBoost Fold 2 -----
CatBoost Fold 2 RMSE: 4.1005

----- CatBoost Fold 3 -----
CatBoost Fold 3 RMSE: 4.1217

----- CatBoost Fold 4 -----


KeyboardInterrupt: 

In [ ]:
# Train final CatBoost model on full data
final_cat_model = CatBoostRegressor(
    loss_function="RMSE",
    n_estimators=1000,
    learning_rate=0.05,
    depth=6,      # again, can try 6 if you want to match teammate
    subsample=0.8,
    random_seed=42,
    verbose=False
)

final_cat_model.fit(
    X,
    y,
    cat_features=cat_features,
    verbose=False
)

# Predict on test_df built from preprocess_pipeline
cat_test_preds = final_cat_model.predict(test_df)

cat_submission = pd.DataFrame({
    id_col: test[id_col],
    target: cat_test_preds
})

# save_submission(cat_submission, run_name="felipe_catboost_farmcluster_lactation")
print("submission_catboost.csv created!")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4625
[LightGBM] [Info] Number of data points in the train set: 209926, number of used features: 47
[LightGBM] [Info] Start training from score 15.595179
Final CatBoost TRAIN RMSE:  3.9662
Final LightGBM TRAIN RMSE:  3.8379
Ensemble(0.7 CB, 0.3 LGB) TRAIN RMSE: 3.9232
Saved submission -> /Users/felipebenitez/ML-Project/submissions/felipe_catboost_only__2025-11-15__22-57-41.csv


# END CATBOOST

# XGBOOST

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# submission functionality
# ------------------------
import os, sys
sys.path.append(os.path.abspath("..")) # so src/ is on the path

import importlib
import submission_utils
importlib.reload(submission_utils) # force reload latest code

from submission_utils import save_submission
# ------------------------
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

In [68]:
# Load data
train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

In [69]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

def preprocess_pipeline_xgb(df, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()

    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col is not None and target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col].copy()
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].astype(str).str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )

    # -----------------------
    # 3) Fill remaining numeric NaNs
    # -----------------------
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    if len(numeric_cols) > 0:
        milk_features[numeric_cols] = milk_features[numeric_cols].fillna(
            milk_features[numeric_cols].median()
        )


    
    

    # -----------------------
    # 4) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"], errors="coerce")
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])

    
    
    

    # -----------------------
    # 5) Farm clustering (guarded)
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        if len(farm_numeric_cols) > 0:
            farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()
            scaler = StandardScaler()
            farm_scaled = scaler.fit_transform(farm_features)

            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
            farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

            milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])
        else:
            milk_features["Farm_Cluster"] = 0


    

        # -----------------------
    # 5.5) Health / management summaries
    # -----------------------
    vax_cols = [c for c in milk_features.columns if c.endswith("_Vaccine")]
    if len(vax_cols) > 0:
        milk_features["Vax_Sum"] = milk_features[vax_cols].sum(axis=1)


    # -----------------------
    # 5.9) Parity categories
    # -----------------------
    if "Parity" in milk_features.columns:
        milk_features["First_Calf"] = (milk_features["Parity"] == 1).astype(int)
        milk_features["Prime_Cow"] = (
            (milk_features["Parity"] >= 2) & (milk_features["Parity"] <= 4)
        ).astype(int)
        milk_features["Old_Cow"] = (milk_features["Parity"] > 4).astype(int)
    

    
    

    # -----------------------
    # 6) Drop only raw row ID (keep Farm_ID for target encoding)
    # -----------------------
    milk_features = milk_features.drop(columns=["Cattle_ID"], errors="ignore")

    # Ensure Farm_Cluster exists
    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y

In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

def add_target_encoding_features(train_df, test_df, y_train,
                                 target_col_name="Milk_Yield_L",
                                 n_splits=5,
                                 random_state=42):
    """
    Adds KFold target-encoded features to train_df and global-mean encodings to test_df.
    Specifically encodes Farm_ID -> Farm_Performance.
    
    Assumes:
      - train_df contains 'Farm_ID'
      - y_train is aligned with train_df (same order, same length)
    """

    # Work on copies
    train_encoded = train_df.copy()
    test_encoded  = test_df.copy()

    # Safety check
    if "Farm_ID" not in train_encoded.columns:
        raise ValueError("Farm_ID column is missing from train_df. "
                         "Make sure preprocess_pipeline_xgb keeps Farm_ID.")

    # Temporary frame for fold splitting
    train_temp = train_encoded.copy()
    train_temp[target_col_name] = y_train.values

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Initialize new column
    train_encoded["Farm_Performance"] = 0.0

    global_avg = train_temp[target_col_name].mean()

    # --- K-fold encoding for TRAIN ---
    for tr_idx, val_idx in kf.split(train_temp):
        X_tr  = train_temp.iloc[tr_idx]
        X_val = train_temp.iloc[val_idx]

        means = X_tr.groupby("Farm_ID")[target_col_name].mean()
        train_encoded.loc[val_idx, "Farm_Performance"] = (
            X_val["Farm_ID"].map(means).fillna(global_avg)
        )

    # --- Global encoding for TEST ---
    global_means = train_temp.groupby("Farm_ID")[target_col_name].mean()
    test_encoded["Farm_Performance"] = (
        test_encoded["Farm_ID"].map(global_means).fillna(global_avg)
    )

    # Drop raw Farm_ID after encoding (prevents huge OHE)
    train_encoded = train_encoded.drop(columns=["Farm_ID"], errors="ignore")
    test_encoded  = test_encoded.drop(columns=["Farm_ID"], errors="ignore")

    return train_encoded, test_encoded

In [71]:
# Load data (same paths you're using)
train = pd.read_csv("../../data/cattle_data_train.csv")
test  = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"

# 1) Base preprocessing (keeps Farm_ID/Breed as strings)
X, y = preprocess_pipeline_xgb(train, target_col=target)
test_df, _ = preprocess_pipeline_xgb(test, target_col=None)

# Reset indices (prevents fold mapping bugs)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# 2) Target encoding (Farm_Performance)
# IMPORTANT: this function must already exist in a previous cell
X, test_df = add_target_encoding_features(X, test_df, y)





# -----------------------
# Post-encoding farm deltas (high signal)
# -----------------------
for df_ in [X, test_df]:
    if "Previous_Week_Avg_Yield" in df_.columns and "Farm_Performance" in df_.columns:
        df_["Prev_vs_Farm"] = df_["Previous_Week_Avg_Yield"] - df_["Farm_Performance"]
        df_["Prev_over_Farm"] = df_["Previous_Week_Avg_Yield"] / (df_["Farm_Performance"] + 1e-6)


    



print("Added Farm_Performance. Now applying One-Hot Encoding...")

# 3) One-hot encoding for remaining object cols (Breed, Climate_Zone, etc.)
X = pd.get_dummies(X, drop_first=False)
test_df = pd.get_dummies(test_df, drop_first=False)

# 4) Align columns so train/test match exactly
X, test_df = X.align(test_df, join="left", axis=1, fill_value=0)

print("Final Train Shape:", X.shape)
print("Final Test Shape:", test_df.shape)

Added Farm_Performance. Now applying One-Hot Encoding...
Final Train Shape: (209926, 68)
Final Test Shape: (40000, 68)


In [72]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_rmses = []
xgb_models = []

print("\nStarting XGBoost Baseline CV (CPU only, xgb.train)...")

# params that mirror what you had in XGBRegressor
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "learning_rate": 0.03,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "min_child_weight": 1.0,
    "tree_method": "hist",   # fast CPU
    "seed": 42,
}

num_boost_round = 2000
early_stop = 80

for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

    dtr  = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params=params,
        dtrain=dtr,
        num_boost_round=num_boost_round,
        evals=[(dval, "val")],
        early_stopping_rounds=early_stop,
        verbose_eval=False
    )

    # Predict using best_iteration safely across versions
    best_iter = model.best_iteration if hasattr(model, "best_iteration") else num_boost_round - 1
    preds = model.predict(dval, iteration_range=(0, best_iter + 1))

    rmse = float(np.sqrt(mean_squared_error(y_val, preds)))
    xgb_rmses.append(rmse)
    xgb_models.append(model)

    print(f"Fold {fold} RMSE: {rmse:.6f}, best_iter={best_iter}")

print("\n==========================")
print(f"XGBoost Average RMSE: {np.mean(xgb_rmses):.6f}")
print("==========================\n")


Starting XGBoost Baseline CV (CPU only, xgb.train)...
Fold 1 RMSE: 4.115534, best_iter=393
Fold 2 RMSE: 4.108522, best_iter=416
Fold 3 RMSE: 4.129335, best_iter=381
Fold 4 RMSE: 4.114713, best_iter=383
Fold 5 RMSE: 4.105947, best_iter=383

XGBoost Average RMSE: 4.114810



# END XGBOOST

In [ ]:
# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)

    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    models.append(model)

print("\n==========================")
print(f"Average RMSE: {np.mean(fold_rmse):.4f}")
print("==========================\n")

LightGBM categorical features: ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type']

----- LightGBM Fold 1 -----
LightGBM Fold 1 RMSE: 5.3580
  -> best_iteration: 45

----- LightGBM Fold 2 -----
LightGBM Fold 2 RMSE: 5.3416
  -> best_iteration: 7

----- LightGBM Fold 3 -----
LightGBM Fold 3 RMSE: 5.3441
  -> best_iteration: 1

----- LightGBM Fold 4 -----
LightGBM Fold 4 RMSE: 5.3530
  -> best_iteration: 21

----- LightGBM Fold 5 -----
LightGBM Fold 5 RMSE: 5.3188
  -> best_iteration: 32

LightGBM Average RMSE: 5.3431



In [ ]:
# --- Reuse already-loaded train/test if they exist ---
# If not, load again
try:
    train
    test
    print("Reusing loaded train/test...")
except NameError:
    print("Loading data fresh...")
    train = pd.read_csv("../../data/cattle_data_train.csv")
    test = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"

def preprocess(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype("category").cat.codes
    return df

train_prep = preprocess(train.drop(columns=[target]))
test_prep = preprocess(test)

X = train_prep
y = train[target]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
xgb_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- XGBoost Fold {fold+1} -----")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = xgb.XGBRegressor(
        n_estimators=700,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",      # fast for large data
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)
    
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    xgb_models.append(model)

print("\n==========================")
print(f"XGBoost Average RMSE = {np.mean(fold_rmse):.4f}")
print("==========================\n")

# --- Train final model on all data ---
final_xgb = xgb.XGBRegressor(
    n_estimators=700,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist",
    random_state=42
)

final_xgb.fit(X, y)

# --- Submission ---
test_preds = final_xgb.predict(test_prep)

submission_xgb = pd.DataFrame({
    id_col: test[id_col],
    target: test_preds
})

# save_submission(submission, run_name="felipe_model")
print("submission_xgb.csv created!")

Reusing loaded train/test...

----- XGBoost Fold 1 -----
Fold 1 RMSE: 4.1888

----- XGBoost Fold 2 -----
Fold 2 RMSE: 4.1517

----- XGBoost Fold 3 -----
Fold 3 RMSE: 4.1568

----- XGBoost Fold 4 -----
Fold 4 RMSE: 4.1814

----- XGBoost Fold 5 -----
Fold 5 RMSE: 4.1694

XGBoost Average RMSE = 4.1696

submission_xgb.csv created!


In [89]:
import pandas as pd
import numpy as np

train = pd.read_csv("../../data/cattle_data_train.csv")

print("="*60)
print("DATASET STRUCTURE ANALYSIS")
print("="*60)

# Date analysis
print("\n1. DATE PATTERNS:")
train['Date'] = pd.to_datetime(train['Date'])
print(f"Date range: {train['Date'].min()} to {train['Date'].max()}")
print(f"Unique dates: {train['Date'].nunique()}")
print(f"Total rows: {len(train)}")
print(f"\nDates per cattle (sample 5):")
print(train.groupby('Cattle_ID')['Date'].nunique().head())

# Farm analysis
print("\n2. FARM PATTERNS:")
print(f"Unique farms: {train['Farm_ID'].nunique()}")
print(f"Avg rows per farm: {len(train) / train['Farm_ID'].nunique():.1f}")
print(f"\nTop 5 farms by count:")
print(train['Farm_ID'].value_counts().head())

# Breed analysis
print("\n3. BREED PATTERNS:")
print(train['Breed'].value_counts())

# Climate analysis
print("\n4. CLIMATE × BREED:")
print(pd.crosstab(train['Climate_Zone'], train['Breed']))

# Correlation with target
print("\n5. TOP CORRELATIONS WITH TARGET:")
numeric_cols = train.select_dtypes(include=[np.number]).columns
corr = train[numeric_cols].corr()['Milk_Yield_L'].sort_values(ascending=False)
print(corr.head(10))

print("\n6. NEGATIVE CORRELATIONS:")
print(corr.tail(10))

DATASET STRUCTURE ANALYSIS

1. DATE PATTERNS:
Date range: 2022-01-01 00:00:00 to 2024-12-30 00:00:00
Unique dates: 1095
Total rows: 210000

Dates per cattle (sample 5):
Cattle_ID
CATTLE_000001    1
CATTLE_000002    1
CATTLE_000003    1
CATTLE_000004    1
CATTLE_000005    1
Name: Date, dtype: int64

2. FARM PATTERNS:
Unique farms: 1000
Avg rows per farm: 210.0

Top 5 farms by count:
Farm_ID
FARM_0842    252
FARM_0724    251
FARM_0405    250
FARM_0571    250
FARM_0937    249
Name: count, dtype: int64

3. BREED PATTERNS:
Breed
Holstein        104775
Jersey           42183
Guernsey         31672
Brown Swiss      31155
Holstien           112
 Brown Swiss        57
Brown Swiss         46
Name: count, dtype: int64

4. CLIMATE × BREED:
Breed           Brown Swiss  Brown Swiss  Brown Swiss   Guernsey  Holstein  \
Climate_Zone                                                                 
Arid                     12         5235             7      5190     17428   
Continental               5 

In [ ]:
import os, sys
import importlib

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb      # still imported in case you need it later
import xgboost as xgb       # same
from catboost import CatBoostRegressor

import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import optuna  # make sure you have `pip install optuna` first


# ======================================================
# 0) Load data
# ======================================================

train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) Your existing preprocessing pipeline (UNCHANGED)
# ======================================================

def preprocess_pipeline(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()
    
    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )
    
    # Fill any remaining numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    # -----------------------
    # 3) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])
    
    # -----------------------
    # 5) Farm clustering (your current version)
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        # Use only numeric features for clustering (exclude IDs)
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        # Aggregate per farm
        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()

        # Scale and cluster farms
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

        # Map back to rows
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    # -----------------------
    # 6) Drop raw IDs
    # -----------------------
    drop_cols = ["Cattle_ID", "Farm_ID"]
    milk_features = milk_features.drop(columns=[c for c in drop_cols if c in milk_features.columns])

    # -----------------------
    # 7) Optional one-hot encoding (we'll keep it OFF for CatBoost)
    # -----------------------
    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    # Make sure Farm_Cluster exists even if something went weird
    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    final_df = milk_features.copy()
    return final_df, y


# ======================================================
# 2) Build X, y, test_df (frozen pipeline)
# ======================================================

X, y = preprocess_pipeline(train, encode_flag=False, target_col=target)
test_df, _ = preprocess_pipeline(test, encode_flag=False, target_col=None)

print("Train shape:", X.shape)
print("Test shape:", test_df.shape)

# Categorical columns for CatBoost (from the processed X)
cat_features = X.select_dtypes(include="object").columns.tolist()
print("CatBoost categorical features:", cat_features)

# KFold for everything
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# ======================================================
# 3) Optuna hyperparameter search (5-fold CV)
# ======================================================

def objective(trial):
    params = {
        "depth": trial.suggest_int("depth", 5, 7),
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.04),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 7.0),
        "subsample": trial.suggest_float("subsample", 0.7, 0.9),
        "random_strength": trial.suggest_float("random_strength", 0.5, 5.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        # you can add more (e.g., border_count) if desired
    }

    fold_rmses = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=3000,            # big cap, rely on early stopping
            early_stopping_rounds=100,
            random_seed=42,
            thread_count=4,
            verbose=False,                # keep Optuna runs quiet
            **params
        )

        model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            cat_features=cat_features,   # using column names
            verbose=False
        )

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_rmses.append(rmse)

    mean_rmse = float(np.mean(fold_rmses))
    return mean_rmse


# You can bump this if you want; 40–60 is a nice compromise
N_TRIALS = 40  

print("\nStarting Optuna hyperparameter search...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=N_TRIALS, n_jobs=2, show_progress_bar=True)

best_params = study.best_params
best_cv_rmse = study.best_value

print("\nOptuna search complete.")
print("Best CV RMSE from Optuna:", best_cv_rmse)
print("Best params:", best_params)


# ======================================================
# 4) Final 5-fold CV ensemble with best_params
# ======================================================

cv_models = []
fold_rmses = []
fold_best_iters = []

print("\nTraining final 5-fold ensemble with best_params...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n----- Final CV Fold {fold} -----")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # use SAME seed as Optuna for consistency
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,
        random_seed=42,
        thread_count=4,
        verbose=False,
        **best_params
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    preds_val = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    best_iter = model.get_best_iteration()

    print(f"Fold {fold} RMSE: {rmse:.4f}, best_iter={best_iter}")

    fold_rmses.append(rmse)
    fold_best_iters.append(best_iter)
    cv_models.append(model)

final_cv_rmse = float(np.mean(fold_rmses))
print("\n===============================")
print(f"Final 5-fold ensemble CV RMSE: {final_cv_rmse:.4f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)

# sanity check vs Optuna's own CV
if final_cv_rmse > best_cv_rmse + 0.001:
    print("⚠️ WARNING: Final CV RMSE is worse than Optuna's best!")
    print(f"Optuna best: {best_cv_rmse:.4f}, Final CV: {final_cv_rmse:.4f}")
    print("This might indicate some instability / differences in folds.")
print("===============================")


# ======================================================
# 5) CV-ensemble predictions on test_df (SAFE baseline)
# ======================================================

cv_test_preds = np.zeros(len(test_df), dtype=float)

for model in cv_models:
    cv_test_preds += model.predict(test_df)

cv_test_preds /= len(cv_models)

# This is your safest, fully CV-validated prediction vector
sub_cv = pd.DataFrame({
    id_col: test[id_col],
    target: cv_test_preds
})

save_submission(sub_cv, run_name="felipe_catboost_optuna_cv_only")
print("Saved CV-only submission: felipe_catboost_optuna_cv_only")


# ======================================================
# 6) Optional: full-data multi-seed ensemble
# ======================================================

avg_best_iter = int(np.mean(fold_best_iters))
print("\nAverage best_iteration across folds:", avg_best_iter)

n_estimators_full = avg_best_iter
print("Using n_estimators for full-data models:", n_estimators_full)

seed_list = [101, 202, 303, 404, 505]
full_seed_models = []
full_seed_preds = np.zeros(len(test_df), dtype=float)

for seed in seed_list:
    print(f"\nTraining full-data model with seed={seed}...")
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=n_estimators_full,
        random_seed=seed,
        thread_count=4,
        verbose=False,
        **best_params
        # no early_stopping_rounds and no eval_set here
    )

    model.fit(
        X,
        y,
        cat_features=cat_features,
        verbose=False
    )

    full_seed_models.append(model)
    full_seed_preds += model.predict(test_df)

full_seed_preds /= len(seed_list)

sub_full = pd.DataFrame({
    id_col: test[id_col],
    target: full_seed_preds
})

save_submission(sub_full, run_name="felipe_catboost_optuna_full_seed_only")
print("Saved full-data seed ensemble submission: felipe_catboost_optuna_full_seed_only")


# ======================================================
# 7) Blended submissions (CV + full-data)
# ======================================================

# 70% CV ensemble, 30% full-data ensemble
alpha_70 = 0.7
blend_70_30 = alpha_70 * cv_test_preds + (1.0 - alpha_70) * full_seed_preds

sub_blend_70_30 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_70_30
})
save_submission(sub_blend_70_30, run_name="felipe_catboost_optuna_blend_70_30")
print("Saved blend 70/30 submission: felipe_catboost_optuna_blend_70_30")

# 50% / 50% blend
alpha_50 = 0.5
blend_50_50 = alpha_50 * cv_test_preds + (1.0 - alpha_50) * full_seed_preds

sub_blend_50_50 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_50_50
})
save_submission(sub_blend_50_50, run_name="felipe_catboost_optuna_blend_50_50")
print("Saved blend 50/50 submission: felipe_catboost_optuna_blend_50_50")


# ======================================================
# 8) EXTRA: OOF RMSE comparison of strategies
#     (CV-only vs "full-seed style" vs blends)
# ======================================================

print("\n\n=== Running OOF comparison for strategies (this is offline, NOT Kaggle) ===")

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# OOF containers (same length as y)
cv_only_oof        = np.zeros(len(X), dtype=float)
full_style_oof     = np.zeros(len(X), dtype=float)
blend_70_30_oof    = np.zeros(len(X), dtype=float)
blend_50_50_oof    = np.zeros(len(X), dtype=float)

# Re-use same KFold splits and same cv_models (order aligned with enumerate start=1)
for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n[OOF Eval] Fold {fold}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # 1) CV-only predictions (from already trained cv_models)
    cv_model = cv_models[fold - 1]
    cv_preds_val = cv_model.predict(X_val)

    # 2) "Full-seed style" ensemble trained ONLY on X_train (no leakage)
    fold_full_preds = np.zeros(len(val_idx), dtype=float)
    for seed in seed_list:
        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=n_estimators_full,
            random_seed=seed,
            verbose=False,
            thread_count=4,
            **best_params
        )
        model.fit(
            X_train,
            y_train,
            cat_features=cat_features,
            verbose=False
        )
        fold_full_preds += model.predict(X_val)

    fold_full_preds /= len(seed_list)

    # Store OOF preds
    cv_only_oof[val_idx]     = cv_preds_val
    full_style_oof[val_idx]  = fold_full_preds
    blend_70_30_oof[val_idx] = alpha_70 * cv_preds_val + (1.0 - alpha_70) * fold_full_preds
    blend_50_50_oof[val_idx] = alpha_50 * cv_preds_val + (1.0 - alpha_50) * fold_full_preds

# Compute OOF RMSEs for each strategy
rmse_cv_only     = rmse(y, cv_only_oof)
rmse_full_style  = rmse(y, full_style_oof)
rmse_blend_70_30 = rmse(y, blend_70_30_oof)
rmse_blend_50_50 = rmse(y, blend_50_50_oof)

print("\n=== OOF RMSE summary (lower is better) ===")
print(f"CV-only ensemble OOF RMSE:        {rmse_cv_only:.6f}")
print(f"Full-seed-style ensemble OOF RMSE: {rmse_full_style:.6f}")
print(f"Blend 70% CV / 30% full OOF RMSE: {rmse_blend_70_30:.6f}")
print(f"Blend 50% CV / 50% full OOF RMSE: {rmse_blend_50_50:.6f}")
print("Note: CV-only OOF RMSE should closely match 'Final 5-fold ensemble CV RMSE' above.")

/Users/felipebenitez/ML-Project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-17 18:36:04,762] A new study created in memory with name: no-name-1a8eea34-828a-47e9-9eb1-d7daa167ceb6


Train shape: (209926, 41)
Test shape: (40000, 41)
CatBoost categorical features: ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type']

Starting Optuna hyperparameter search...


Best trial: 0. Best value: 4.10664:   2%|▎         | 1/40 [04:02<2:37:25, 242.19s/it]

[I 2025-11-17 18:40:06,955] Trial 0 finished with value: 4.1066350616145835 and parameters: {'depth': 7, 'learning_rate': 0.030033228718532787, 'l2_leaf_reg': 1.7953972301724823, 'subsample': 0.7796965026065017, 'random_strength': 1.0696793682056738, 'bagging_temperature': 0.9530309362394171}. Best is trial 0 with value: 4.1066350616145835.


Best trial: 0. Best value: 4.10664:   5%|▌         | 2/40 [04:12<1:07:03, 105.88s/it]

...

[I 2025-11-17 20:06:24,329] Trial 37 finished with value: 4.106685214153209 and parameters: {'depth': 6, 'learning_rate': 0.021224595425353376, 'l2_leaf_reg': 3.7288952126765063, 'subsample': 0.8079377679075879, 'random_strength': 2.0751816513193564, 'bagging_temperature': 0.20718472942570026}. Best is trial 29 with value: 4.106391174299612.


Best trial: 29. Best value: 4.10639:  98%|█████████▊| 39/40 [1:33:43<02:50, 170.24s/it]

[I 2025-11-17 20:09:48,357] Trial 38 finished with value: 4.107314015070822 and parameters: {'depth': 5, 'learning_rate': 0.024778472112600383, 'l2_leaf_reg': 3.805345305329153, 'subsample': 0.8100157870165497, 'random_strength': 1.9927500915285554, 'bagging_temperature': 0.2433839797830411}. Best is trial 29 with value: 4.106391174299612.


Best trial: 29. Best value: 4.10639: 100%|██████████| 40/40 [1:34:29<00:00, 141.73s/it]


[I 2025-11-17 20:10:33,822] Trial 39 finished with value: 4.107113798299714 and parameters: {'depth': 5, 'learning_rate': 0.02733868841994063, 'l2_leaf_reg': 5.343837851160658, 'subsample': 0.7589913249369241, 'random_strength': 0.8732091640464774, 'bagging_temperature': 0.5614393962636348}. Best is trial 29 with value: 4.106391174299612.

Optuna search complete.
Best CV RMSE from Optuna: 4.106391174299612
Best params: {'depth': 6, 'learning_rate': 0.022872270426979868, 'l2_leaf_reg': 4.014881862534349, 'subsample': 0.847236734294946, 'random_strength': 0.731107887920747, 'bagging_temperature': 0.4635035306948895}

Training final 5-fold ensemble with best_params...

----- Final CV Fold 1 -----
Fold 1 RMSE: 4.1070, best_iter=1157

----- Final CV Fold 2 -----
Fold 2 RMSE: 4.1002, best_iter=1127

----- Final CV Fold 3 -----
Fold 3 RMSE: 4.1201, best_iter=1093

----- Final CV Fold 4 -----
Fold 4 RMSE: 4.1063, best_iter=1094

----- Final CV Fold 5 -----
Fold 5 RMSE: 4.0984, best_iter=999

F

In [ ]:
import os, sys
import importlib

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb      # still imported in case you need it later
import xgboost as xgb       # same
from catboost import CatBoostRegressor

import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import optuna  # make sure you have `pip install optuna` first


# ======================================================
# 0) Load data
# ======================================================

train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) Your existing preprocessing pipeline (UNCHANGED)
# ======================================================

def preprocess_pipeline(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()
    
    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )
    


            

    # Fill any remaining numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    # -----------------------
    # 3) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])
    
    

    # -----------------------
    # 5) Farm clustering
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        # Use only numeric features for clustering (exclude IDs)
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        # Aggregate per farm
        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()

        # Scale and cluster farms
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

        # Map back to rows
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])


    # -----------------------
    # INSERT: Weight_Efficiency_Z (Breed + Age_Year)
    # -----------------------
    if "Weight_kg" in milk_features.columns and "Age_Months" in milk_features.columns:
        milk_features["Age_Year"] = milk_features["Age_Months"] // 12
        group_cols = ["Breed", "Age_Year"]

        # Calculate cohort stats using transform so it maps back to original rows
        cohort_mean = milk_features.groupby(group_cols)["Weight_kg"].transform("mean")
        cohort_std = milk_features.groupby(group_cols)["Weight_kg"].transform("std")

        # Calculate Z-Score: How heavy is this cow relative to her peers?
        milk_features["Weight_Efficiency_Z"] = (
            (milk_features["Weight_kg"] - cohort_mean) / (cohort_std + 1e-5)
        )

        milk_features = milk_features.drop(columns=["Age_Year"])

    # -----------------------
    # 6) Drop raw IDs
    # -----------------------
    drop_cols = ["Cattle_ID"]
    milk_features = milk_features.drop(columns=[c for c in drop_cols if c in milk_features.columns])

    # -----------------------
    # 7) Optional one-hot encoding (we'll keep it OFF for CatBoost)
    # -----------------------
    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    # Make sure Farm_Cluster exists even if something went weird
    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    final_df = milk_features.copy()
    return final_df, y


# ======================================================
# 2) Build X, y, test_df (frozen pipeline)
# ======================================================

X, y = preprocess_pipeline(train, encode_flag=False, target_col=target)
test_df, _ = preprocess_pipeline(test, encode_flag=False, target_col=None)

print("Train shape:", X.shape)
print("Test shape:", test_df.shape)



# ==============================================================================
# REPLACEMENT: Farm_Performance Target Encoding (The Missing Signal)
# ==============================================================================
from sklearn.model_selection import KFold

# FIX: Reset indices (Keep this from previous attempt, it worked)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
test_df = test_df.reset_index(drop=True) 

def add_target_encoding_features(train_df, test_df, y_train, target_col_name="Milk_Yield_L", n_splits=5):
    # CHANGE: Encode Farm_ID instead of Bio features
    # This captures "Managerial Quality" - the specific effect of this farm on yield
    encodings = {
        "Farm_Performance": ["Farm_ID"]
    }
    
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Combine X and y temporarily for the training split
    train_temp = train_encoded.copy()
    train_temp[target_col_name] = y_train.values

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for new_col, group_cols in encodings.items():
        # --- A) TRAINING SET: K-Fold Encoding (Prevents Leakage) ---
        train_encoded[new_col] = 0.0
        
        for tr_ind, val_ind in kf.split(train_temp):
            X_tr, X_val = train_temp.iloc[tr_ind], train_temp.iloc[val_ind]
            
            # Calculate mean yield per Farm on the 'training' part
            means = X_tr.groupby(group_cols)[target_col_name].mean()
            
            # Map to the 'validation' part
            # Since it's just one column (Farm_ID), simple map works
            train_encoded.loc[val_ind, new_col] = X_val[group_cols[0]].map(means).fillna(X_tr[target_col_name].mean())

        # --- B) TEST SET: Global Mean Encoding ---
        # Compute global farm stats from full training data
        global_means = train_temp.groupby(group_cols)[target_col_name].mean()
        global_avg = train_temp[target_col_name].mean()
        
        # Map to test set
        test_encoded[new_col] = test_encoded[group_cols[0]].map(global_means).fillna(global_avg)

    # OPTIONAL: Drop the raw Farm_ID now that we have the encoded score
    # This keeps the model clean
    train_encoded = train_encoded.drop(columns=["Farm_ID"], errors='ignore')
    test_encoded = test_encoded.drop(columns=["Farm_ID"], errors='ignore')

    return train_encoded, test_encoded

# Execute
X, test_df = add_target_encoding_features(X, test_df, y)
print("Added feature: Farm_Performance")

# ... Continue to CatBoost training ...



# Categorical columns for CatBoost (from the processed X)
cat_features = X.select_dtypes(include="object").columns.tolist()
print("CatBoost categorical features:", cat_features)

# KFold for everything
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# ======================================================
# 3) Optuna hyperparameter search (5-fold CV)
# ======================================================

def objective(trial):
    params = {
        "depth": 6,
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.035),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2.0, 7.0),
        "random_strength": trial.suggest_float("random_strength", 0.1, 5.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        # you can add more (e.g., border_count) if desired
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 60)
    }

    fold_rmses = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=3000,            # big cap, rely on early stopping
            early_stopping_rounds=100,
            random_seed=42,
            thread_count=5,
            verbose=False,                # keep Optuna runs quiet
            **params
        )

        model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            cat_features=cat_features,   # using column names
            verbose=False
        )

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_rmses.append(rmse)

    mean_rmse = float(np.mean(fold_rmses))
    return mean_rmse


# You can bump this if you want; 40–60 is a nice compromise
N_TRIALS = 40  

print("\nStarting Optuna hyperparameter search...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=N_TRIALS, n_jobs=4, show_progress_bar=True)

best_params = study.best_params
best_params["depth"] = 6   # hard lock depth
best_cv_rmse = study.best_value

print("\nOptuna search complete.")
print("Best CV RMSE from Optuna:", best_cv_rmse)
print("Best params:", best_params)


# ======================================================
# 4) Final 5-fold CV ensemble with best_params
# ======================================================

cv_models = []
fold_rmses = []
fold_best_iters = []

print("\nTraining final 5-fold ensemble with best_params...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n----- Final CV Fold {fold} -----")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # use SAME seed as Optuna for consistency
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,
        random_seed=42,
        thread_count=5,
        verbose=False,
        task_type="GPU",
        devices="0",   # or "0:1" if he wants multi-GPU (rarely needed)
        **best_params
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    preds_val = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    best_iter = model.get_best_iteration()

    print(f"Fold {fold} RMSE: {rmse:.4f}, best_iter={best_iter}")

    fold_rmses.append(rmse)
    fold_best_iters.append(best_iter)
    cv_models.append(model)

final_cv_rmse = float(np.mean(fold_rmses))
print("\n===============================")
print(f"Final 5-fold ensemble CV RMSE: {final_cv_rmse:.4f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)

# sanity check vs Optuna's own CV
if final_cv_rmse > best_cv_rmse + 0.001:
    print("⚠️ WARNING: Final CV RMSE is worse than Optuna's best!")
    print(f"Optuna best: {best_cv_rmse:.4f}, Final CV: {final_cv_rmse:.4f}")
    print("This might indicate some instability / differences in folds.")
print("===============================")


# ======================================================
# 5) CV-ensemble predictions on test_df (SAFE baseline)
# ======================================================

cv_test_preds = np.zeros(len(test_df), dtype=float)

for model in cv_models:
    cv_test_preds += model.predict(test_df)

cv_test_preds /= len(cv_models)

# This is your safest, fully CV-validated prediction vector
sub_cv = pd.DataFrame({
    id_col: test[id_col],
    target: cv_test_preds
})

save_submission(sub_cv, run_name="felipe_catboost_optuna_cv_only")
print("Saved CV-only submission: felipe_catboost_optuna_cv_only")


# ======================================================
# 6) Optional: full-data multi-seed ensemble
# ======================================================

avg_best_iter = int(np.mean(fold_best_iters))
print("\nAverage best_iteration across folds:", avg_best_iter)

n_estimators_full = avg_best_iter
print("Using n_estimators for full-data models:", n_estimators_full)

seed_list = [101, 202, 303, 404, 505]
full_seed_models = []
full_seed_preds = np.zeros(len(test_df), dtype=float)

for seed in seed_list:
    print(f"\nTraining full-data model with seed={seed}...")
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=n_estimators_full,
        random_seed=seed,
        thread_count=5,
        verbose=False,
        task_type="GPU",
        devices="0",   # or "0:1" if he wants multi-GPU (rarely needed)
        **best_params
        # no early_stopping_rounds and no eval_set here
    )

    model.fit(
        X,
        y,
        cat_features=cat_features,
        verbose=False
    )

    full_seed_models.append(model)
    full_seed_preds += model.predict(test_df)

full_seed_preds /= len(seed_list)

sub_full = pd.DataFrame({
    id_col: test[id_col],
    target: full_seed_preds
})

save_submission(sub_full, run_name="felipe_catboost_optuna_full_seed_only")
print("Saved full-data seed ensemble submission: felipe_catboost_optuna_full_seed_only")


def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# ======================================================
# 7) Blended submissions (CV + full-data)
# ======================================================

cv_only_oof    = np.zeros(len(X), dtype=float)
full_style_oof = np.zeros(len(X), dtype=float)

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n[OOF Eval] Fold {fold}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # CV-only preds
    cv_model = cv_models[fold - 1]
    cv_preds_val = cv_model.predict(X_val)

    # Full-seed-style preds (train on fold train only)
    fold_full_preds = np.zeros(len(val_idx), dtype=float)
    for seed in seed_list:
        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=n_estimators_full,
            random_seed=seed,
            verbose=False,
            thread_count=5,
            task_type="GPU",
            devices="0",   # or "0:1" if he wants multi-GPU (rarely needed)
            **best_params
        )
        model.fit(X_train, y_train, cat_features=cat_features, verbose=False)
        fold_full_preds += model.predict(X_val)

    fold_full_preds /= len(seed_list)

    cv_only_oof[val_idx]    = cv_preds_val
    full_style_oof[val_idx] = fold_full_preds

rmse_cv_only    = rmse(y, cv_only_oof)
rmse_full_style = rmse(y, full_style_oof)

print("\n=== OOF RMSE (base strategies) ===")
print(f"CV-only OOF RMSE:        {rmse_cv_only:.6f}")
print(f"Full-seed-style OOF RMSE:{rmse_full_style:.6f}")

# Search best alpha
alphas = np.linspace(0, 1, 101)
best_alpha, best_rmse = None, 1e9

for a in alphas:
    blend_oof = a * cv_only_oof + (1 - a) * full_style_oof
    r = rmse(y, blend_oof)
    if r < best_rmse:
        best_rmse, best_alpha = r, a

print(f"\nBest alpha from OOF search: {best_alpha:.2f}")
print(f"Best blended OOF RMSE:      {best_rmse:.6f}")

# Final blended test preds using best_alpha
final_blend_test = best_alpha * cv_test_preds + (1 - best_alpha) * full_seed_preds

sub_blend_best = pd.DataFrame({
    id_col: test[id_col],
    target: final_blend_test
})
save_submission(sub_blend_best, run_name=f"felipe_catboost_optuna_blend_best_{best_alpha:.2f}")
print("Saved best-alpha blended submission.")

print("\n=== OOF RMSE comparison ===")
print(f"CV-only:             {rmse_cv_only:.6f}")
print(f"Full-seed-style:     {rmse_full_style:.6f}")
print(f"Best blend (alpha={best_alpha:.2f}): {best_rmse:.6f}")


# NEW TUNING + ENSEMBLING (WITHOUT HARDCODED BLENDS) FOR ORIGINAL CATBOOST

In [33]:
import os, sys
import importlib

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb      # still imported in case you need it later
import xgboost as xgb       # same
from catboost import CatBoostRegressor

import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import optuna  # make sure you have `pip install optuna` first


# ======================================================
# 0) Load data
# ======================================================

train = pd.read_csv("../../data/cattle_data_train.csv")
test = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) Your existing preprocessing pipeline (UNCHANGED)
# ======================================================

def preprocess_pipeline(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()
    
    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )
    
    # Fill any remaining numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    # -----------------------
    # 3) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])
    
    # -----------------------
    # 5) Farm clustering (your current version)
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        # Use only numeric features for clustering (exclude IDs)
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        # Aggregate per farm
        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()

        # Scale and cluster farms
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

        # Map back to rows
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    # -----------------------
    # 6) Drop raw IDs
    # -----------------------
    drop_cols = ["Cattle_ID", "Farm_ID"]
    milk_features = milk_features.drop(columns=[c for c in drop_cols if c in milk_features.columns])

    # -----------------------
    # 7) Optional one-hot encoding (we'll keep it OFF for CatBoost)
    # -----------------------
    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    # Make sure Farm_Cluster exists even if something went weird
    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    final_df = milk_features.copy()
    return final_df, y


# ======================================================
# 2) Build X, y, test_df (frozen pipeline)
# ======================================================

X, y = preprocess_pipeline(train, encode_flag=False, target_col=target)
test_df, _ = preprocess_pipeline(test, encode_flag=False, target_col=None)

print("Train shape:", X.shape)
print("Test shape:", test_df.shape)

# Categorical columns for CatBoost (from the processed X)
cat_features = X.select_dtypes(include="object").columns.tolist()
print("CatBoost categorical features:", cat_features)

# KFold for everything
kf = KFold(n_splits=5, shuffle=True, random_state=42)


fixed_params = {
    "depth": 6,
    "bootstrap_type": "Bayesian",
    "grow_policy": "SymmetricTree",
}

# ======================================================
# 3) Optuna hyperparameter search (5-fold CV)
# ======================================================

def objective(trial):
    tuned_params = {
        # 2. QUANTIZATION (S-Tier Tweak)
        # 128 is a sweet spot for noisy data (vs default 254)
        "border_count": trial.suggest_categorical("border_count", [128, 254]),

        # 3. REGULARIZATION
        "learning_rate": trial.suggest_float("learning_rate", 0.015, 0.06),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "random_strength": trial.suggest_float("random_strength", 0.1, 8.0),

        # 4. BAYESIAN BAGGING (Replaces subsample)
        # 0 = standard weighting, >1 = adds more randomness to weights
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 3.0),

        # 5. COLUMN SAMPLING
        "rsm": trial.suggest_float("rsm", 0.7, 1.0),  # colsample_bylevel

        # 6. OVERFITTING CONTROL
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 80),
    }

    params = {**fixed_params, **tuned_params}

    fold_rmses = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=3000,            # big cap, rely on early stopping
            early_stopping_rounds=100,
            random_seed=42,
            use_best_model=True,
            thread_count=4,
            verbose=False,                # keep Optuna runs quiet
            **params
        )

        model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            cat_features=cat_features,   # using column names
            verbose=False
        )

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_rmses.append(rmse)

    mean_rmse = float(np.mean(fold_rmses))
    return mean_rmse


# You can bump this if you want; 40–60 is a nice compromise
N_TRIALS = 80  

print("\nStarting Optuna hyperparameter search...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=N_TRIALS, n_jobs=2, show_progress_bar=True)

best_params = study.best_params
best_cv_rmse = study.best_value

final_params = {**fixed_params, **best_params}

print("\nOptuna search complete.")
print("Best CV RMSE from Optuna:", best_cv_rmse)
print("Best params:", best_params)
print("Final params used for training:", final_params)

import json
with open("catboost_best_params.json", "w") as f:
    json.dump(final_params, f, indent=2)
print("Saved best params to catboost_best_params.json")


# ======================================================
# 4) Final 5-fold CV ensemble with best_params
# ======================================================

cv_models = []
fold_rmses = []
fold_best_iters = []

print("\nTraining final 5-fold ensemble with best_params...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n----- Final CV Fold {fold} -----")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # use SAME seed as Optuna for consistency
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,
        use_best_model=True,
        random_seed=42,
        thread_count=4,
        verbose=False,
        **final_params
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    preds_val = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    best_iter = model.get_best_iteration()

    print(f"Fold {fold} RMSE: {rmse:.4f}, best_iter={best_iter}")

    fold_rmses.append(rmse)
    fold_best_iters.append(best_iter)
    cv_models.append(model)

final_cv_rmse = float(np.mean(fold_rmses))
print("\n===============================")
print(f"Final 5-fold ensemble CV RMSE: {final_cv_rmse:.4f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)

# sanity check vs Optuna's own CV
if final_cv_rmse > best_cv_rmse + 0.001:
    print("⚠️ WARNING: Final CV RMSE is worse than Optuna's best!")
    print(f"Optuna best: {best_cv_rmse:.4f}, Final CV: {final_cv_rmse:.4f}")
    print("This might indicate some instability / differences in folds.")
print("===============================")


# ======================================================
# 5) CV-ensemble predictions on test_df (SAFE baseline)
# ======================================================

cv_test_preds = np.zeros(len(test_df), dtype=float)

for model in cv_models:
    cv_test_preds += model.predict(test_df)

cv_test_preds /= len(cv_models)

# This is your safest, fully CV-validated prediction vector
sub_cv = pd.DataFrame({
    id_col: test[id_col],
    target: cv_test_preds
})

save_submission(sub_cv, run_name="felipe_catboost_optuna_cv_only")
print("Saved CV-only submission: felipe_catboost_optuna_cv_only")


# ======================================================
# 6) Optional: full-data multi-seed ensemble
# ======================================================

avg_best_iter = int(np.mean(fold_best_iters))
print("\nAverage best_iteration across folds:", avg_best_iter)

n_estimators_full = avg_best_iter
print("Using n_estimators for full-data models:", n_estimators_full)

seed_list = [101, 202, 303, 404, 505]
full_seed_models = []
full_seed_preds = np.zeros(len(test_df), dtype=float)

for seed in seed_list:
    print(f"\nTraining full-data model with seed={seed}...")
    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=n_estimators_full,
        random_seed=seed,
        thread_count=4,
        verbose=False,
        **final_params
        # no early_stopping_rounds and no eval_set here
    )

    model.fit(
        X,
        y,
        cat_features=cat_features,
        verbose=False
    )

    full_seed_models.append(model)
    full_seed_preds += model.predict(test_df)

full_seed_preds /= len(seed_list)

sub_full = pd.DataFrame({
    id_col: test[id_col],
    target: full_seed_preds
})

save_submission(sub_full, run_name="felipe_catboost_optuna_full_seed_only")
print("Saved full-data seed ensemble submission: felipe_catboost_optuna_full_seed_only")


# ======================================================
# 7) Blended submissions (CV + full-data)
# ======================================================

# 70% CV ensemble, 30% full-data ensemble
alpha_70 = 0.7
blend_70_30 = alpha_70 * cv_test_preds + (1.0 - alpha_70) * full_seed_preds

sub_blend_70_30 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_70_30
})
save_submission(sub_blend_70_30, run_name="felipe_catboost_optuna_blend_70_30")
print("Saved blend 70/30 submission: felipe_catboost_optuna_blend_70_30")

# 50% / 50% blend
alpha_50 = 0.5
blend_50_50 = alpha_50 * cv_test_preds + (1.0 - alpha_50) * full_seed_preds

sub_blend_50_50 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_50_50
})
save_submission(sub_blend_50_50, run_name="felipe_catboost_optuna_blend_50_50")
print("Saved blend 50/50 submission: felipe_catboost_optuna_blend_50_50")


# ======================================================
# 8) EXTRA: OOF RMSE comparison of strategies
#     (CV-only vs "full-seed style" vs blends)
# ======================================================

print("\n\n=== Running OOF comparison for strategies (this is offline, NOT Kaggle) ===")

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# OOF containers (same length as y)
cv_only_oof        = np.zeros(len(X), dtype=float)
full_style_oof     = np.zeros(len(X), dtype=float)
blend_70_30_oof    = np.zeros(len(X), dtype=float)
blend_50_50_oof    = np.zeros(len(X), dtype=float)

# Re-use same KFold splits and same cv_models (order aligned with enumerate start=1)
for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n[OOF Eval] Fold {fold}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # 1) CV-only predictions (from already trained cv_models)
    cv_model = cv_models[fold - 1]
    cv_preds_val = cv_model.predict(X_val)

    # 2) "Full-seed style" ensemble trained ONLY on X_train (no leakage)
    fold_full_preds = np.zeros(len(val_idx), dtype=float)
    for seed in seed_list:
        model = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=n_estimators_full,
            random_seed=seed,
            verbose=False,
            thread_count=4,
            **final_params
        )
        model.fit(
            X_train,
            y_train,
            cat_features=cat_features,
            verbose=False
        )
        fold_full_preds += model.predict(X_val)

    fold_full_preds /= len(seed_list)

    # Store OOF preds
    cv_only_oof[val_idx]     = cv_preds_val
    full_style_oof[val_idx]  = fold_full_preds
    blend_70_30_oof[val_idx] = alpha_70 * cv_preds_val + (1.0 - alpha_70) * fold_full_preds
    blend_50_50_oof[val_idx] = alpha_50 * cv_preds_val + (1.0 - alpha_50) * fold_full_preds

# Compute OOF RMSEs for each strategy
rmse_cv_only     = rmse(y, cv_only_oof)
rmse_full_style  = rmse(y, full_style_oof)
rmse_blend_70_30 = rmse(y, blend_70_30_oof)
rmse_blend_50_50 = rmse(y, blend_50_50_oof)

print("\n=== OOF RMSE summary (lower is better) ===")
print(f"CV-only ensemble OOF RMSE:        {rmse_cv_only:.6f}")
print(f"Full-seed-style ensemble OOF RMSE: {rmse_full_style:.6f}")
print(f"Blend 70% CV / 30% full OOF RMSE: {rmse_blend_70_30:.6f}")
print(f"Blend 50% CV / 50% full OOF RMSE: {rmse_blend_50_50:.6f}")
print("Note: CV-only OOF RMSE should closely match 'Final 5-fold ensemble CV RMSE' above.")


# ======================================================
# 9) Find best alpha blend by OOF (NOT hardcoded)
# ======================================================

alphas = np.linspace(0, 1, 201)  # 0.005 steps
best_alpha = None
best_alpha_rmse = float("inf")

for a in alphas:
    blend_oof = a * cv_only_oof + (1 - a) * full_style_oof
    r = rmse(y, blend_oof)
    if r < best_alpha_rmse:
        best_alpha_rmse = r
        best_alpha = a

print("\n=== Best OOF blend search ===")
print(f"Best alpha (CV weight) = {best_alpha:.3f}")
print(f"Best blended OOF RMSE  = {best_alpha_rmse:.6f}")

# Use this alpha to blend TEST predictions too
best_blend_test = best_alpha * cv_test_preds + (1 - best_alpha) * full_seed_preds

sub_best_blend = pd.DataFrame({
    id_col: test[id_col],
    target: best_blend_test
})

save_submission(sub_best_blend, run_name=f"felipe_catboost_optuna_best_blend_{best_alpha:.3f}")
print("Saved best-alpha blend submission.")

/Users/felipebenitez/ML-Project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-22 22:05:21,190] A new study created in memory with name: no-name-3fc96153-e158-44e4-9daf-eabe305f1fa1


Train shape: (209926, 41)
Test shape: (40000, 41)
CatBoost categorical features: ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type']

Starting Optuna hyperparameter search...


  0%|          | 0/80 [00:36<?, ?it/s]


KeyboardInterrupt: 

[I 2025-11-22 22:09:02,121] Trial 0 finished with value: 4.110370169885739 and parameters: {'border_count': 128, 'learning_rate': 0.04830379037080595, 'l2_leaf_reg': 3.4068584765597096, 'random_strength': 2.283498470199023, 'bagging_temperature': 2.2866267950920003, 'rsm': 0.838134391147505, 'min_data_in_leaf': 1}. Best is trial 0 with value: 4.110370169885739.


#  TUNING + ENSEMBLING (WITHOUT HARDCODED BLENDS) FOR XGBOOST

In [74]:
import os, sys, importlib, json
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# submission utils
sys.path.append(os.path.abspath(".."))
import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission

# ======================================================
# 0) Load data
# ======================================================
train = pd.read_csv("../../data/cattle_data_train.csv")
test  = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"

# ======================================================
# 1) Preprocessing (keeps Farm_ID for encoding, keeps lb)
# ======================================================
def preprocess_pipeline_xgb(df, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()

    # 1) Drop impossible targets (train only)
    if target_col is not None and target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0].copy()
        y = milk_features[target_col].copy()
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # 2) Basic cleaning
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].astype(str).str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )

    # Fill numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    if len(numeric_cols) > 0:
        milk_features[numeric_cols] = milk_features[numeric_cols].fillna(
            milk_features[numeric_cols].median()
        )

    # 3) Date features (no is_weekend)
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"], errors="coerce")
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])

    # 4) Farm clustering
    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]
        if len(farm_numeric_cols) > 0:
            farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()
            scaler = StandardScaler()
            farm_scaled = scaler.fit_transform(farm_features)
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
            farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)
            milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])
        else:
            milk_features["Farm_Cluster"] = 0

    # 5) Vaccine sum
    vax_cols = [c for c in milk_features.columns if c.endswith("_Vaccine")]
    if len(vax_cols) > 0:
        milk_features["Vax_Sum"] = milk_features[vax_cols].sum(axis=1)

    # 6) Parity categories
    if "Parity" in milk_features.columns:
        milk_features["First_Calf"] = (milk_features["Parity"] == 1).astype(int)
        milk_features["Prime_Cow"] = ((milk_features["Parity"] >= 2) & (milk_features["Parity"] <= 4)).astype(int)
        milk_features["Old_Cow"]   = (milk_features["Parity"] > 4).astype(int)

    # 7) Drop ONLY row ID. Keep Farm_ID for encoding. Keep lb feature.
    milk_features = milk_features.drop(columns=["Cattle_ID"], errors="ignore")

    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y


# ======================================================
# 2) Leak-safe fold target encoding
# ======================================================
def fold_target_encode(train_df, y_train, val_df, test_df,
                       col="Farm_ID", new_name="Farm_Performance"):
    """
    Encodes col using ONLY outer-train targets -> no leakage.
    """
    train_df = train_df.copy()
    val_df   = val_df.copy()
    test_df  = test_df.copy()

    global_avg = float(y_train.mean())
    means = y_train.groupby(train_df[col]).mean()

    train_df[new_name] = train_df[col].map(means).fillna(global_avg)
    val_df[new_name]   = val_df[col].map(means).fillna(global_avg)
    test_df[new_name]  = test_df[col].map(means).fillna(global_avg)

    train_df = train_df.drop(columns=[col], errors="ignore")
    val_df   = val_df.drop(columns=[col], errors="ignore")
    test_df  = test_df.drop(columns=[col], errors="ignore")

    return train_df, val_df, test_df


def add_farm_deltas(df_):
    df_ = df_.copy()
    if "Previous_Week_Avg_Yield" in df_.columns and "Farm_Performance" in df_.columns:
        df_["Prev_vs_Farm"]   = df_["Previous_Week_Avg_Yield"] - df_["Farm_Performance"]
        df_["Prev_over_Farm"] = df_["Previous_Week_Avg_Yield"] / (df_["Farm_Performance"] + 1e-6)
    return df_


# ======================================================
# 3) Build base datasets once
# ======================================================
X_base, y = preprocess_pipeline_xgb(train, target_col=target)
test_base, _ = preprocess_pipeline_xgb(test, target_col=None)

X_base = X_base.reset_index(drop=True)
y = y.reset_index(drop=True)
test_base = test_base.reset_index(drop=True)


# ======================================================
# 4) Diamond-Tier Optuna Tuning (500 Trials + Advanced Space)
#    IMPORTANT: tune on fold-processed + OHE features
# ======================================================
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if grow_policy == "lossguide":
        max_depth = trial.suggest_int("max_depth", 0, 10)
    else:
        max_depth = trial.suggest_int("max_depth", 3, 12)

    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "seed": 42,
        "nthread": 4,
        "grow_policy": grow_policy,

        "max_bin": trial.suggest_categorical("max_bin", [128, 256, 512]),

        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),

        "max_depth": max_depth,
        "max_leaves": trial.suggest_int("max_leaves", 16, 256),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 20.0, log=True),

        "subsample": trial.suggest_float("subsample", 0.5, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.95),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 0.95),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 0.95),

        "gamma": trial.suggest_float("gamma", 1e-4, 10.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 50.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 50.0, log=True),
    }

    fold_rmses = []

    for tr_idx, val_idx in kf.split(X_base):
        X_tr_base = X_base.iloc[tr_idx]
        X_val_base = X_base.iloc[val_idx]
        y_tr = y.iloc[tr_idx]
        y_val = y.iloc[val_idx]

        # fold-safe TE
        X_tr, X_val, _ = fold_target_encode(X_tr_base, y_tr, X_val_base, test_base)
        X_tr  = add_farm_deltas(X_tr)
        X_val = add_farm_deltas(X_val)

        # OHE + align within fold
        X_tr  = pd.get_dummies(X_tr, drop_first=False)
        X_val = pd.get_dummies(X_val, drop_first=False)
        X_tr, X_val = X_tr.align(X_val, join="left", axis=1, fill_value=0)

        dtr  = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtr,
            num_boost_round=3000,
            evals=[(dval, "val")],
            early_stopping_rounds=100,
            verbose_eval=False
        )

        best_iter = model.best_iteration if hasattr(model, "best_iteration") else 2999
        preds = model.predict(dval, iteration_range=(0, best_iter + 1))
        fold_rmses.append(np.sqrt(mean_squared_error(y_val, preds)))

    return float(np.mean(fold_rmses))


N_TRIALS = 500
print(f"\nStarting Diamond-Tier XGBoost Optuna ({N_TRIALS} trials)...")

study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=42)
)
study.optimize(objective, n_trials=N_TRIALS, n_jobs=2, show_progress_bar=True)

best_params = study.best_params
best_params.update({
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "seed": 42,
    "nthread": 4
})
final_params = dict(best_params)

print("\nBest Params:", best_params)
with open("xgboost_diamond_params.json", "w") as f:
    json.dump(best_params, f, indent=4)
print("Saved best params to xgboost_diamond_params.json")


# ======================================================
# 5) Build GLOBAL feature template (prevents mismatch)
# ======================================================
X_full_enc, _, test_full_enc = fold_target_encode(X_base, y, X_base.iloc[:1], test_base)
X_full_enc  = add_farm_deltas(X_full_enc)
test_full_enc = add_farm_deltas(test_full_enc)

X_full_enc = pd.get_dummies(X_full_enc, drop_first=False)
test_full_enc = pd.get_dummies(test_full_enc, drop_first=False)
X_full_enc, test_full_enc = X_full_enc.align(test_full_enc, join="left", axis=1, fill_value=0)

GLOBAL_COLS = X_full_enc.columns.tolist()


# ======================================================
# 6) Final 5-fold CV ensemble with tuned params
# ======================================================
cv_models = []
fold_rmses = []
fold_best_iters = []
cv_oof = np.zeros(len(X_base), dtype=float)

num_boost_round = 6000
early_stop = 120

print("\nTraining final XGB 5-fold ensemble...")

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_base), 1):
    print(f"\n----- Final Fold {fold} -----")
    X_tr_base = X_base.iloc[tr_idx]
    X_val_base = X_base.iloc[val_idx]
    y_tr = y.iloc[tr_idx]
    y_val = y.iloc[val_idx]

    X_tr, X_val, _ = fold_target_encode(X_tr_base, y_tr, X_val_base, test_base)
    X_tr  = add_farm_deltas(X_tr)
    X_val = add_farm_deltas(X_val)

    X_tr  = pd.get_dummies(X_tr, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)
    X_val = pd.get_dummies(X_val, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)

    dtr  = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params=final_params,
        dtrain=dtr,
        num_boost_round=num_boost_round,
        evals=[(dval, "val")],
        early_stopping_rounds=early_stop,
        verbose_eval=False
    )

    best_iter = model.best_iteration if hasattr(model, "best_iteration") else num_boost_round - 1
    preds_val = model.predict(dval, iteration_range=(0, best_iter + 1))

    rmse = float(np.sqrt(mean_squared_error(y_val, preds_val)))
    print(f"Fold {fold} RMSE: {rmse:.6f}, best_iter={best_iter}")

    fold_rmses.append(rmse)
    fold_best_iters.append(best_iter)
    cv_models.append(model)
    cv_oof[val_idx] = preds_val

final_cv_rmse = float(np.sqrt(mean_squared_error(y, cv_oof)))
print("\n===========================")
print(f"Final CV RMSE: {final_cv_rmse:.6f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)
print("===========================\n")


# ======================================================
# 7) CV-ensemble predictions on test
# ======================================================
dtest = xgb.DMatrix(test_full_enc)

cv_test_preds = np.zeros(len(test_full_enc), dtype=float)
for m in cv_models:
    best_iter = m.best_iteration if hasattr(m, "best_iteration") else int(np.mean(fold_best_iters))
    cv_test_preds += m.predict(dtest, iteration_range=(0, best_iter + 1))
cv_test_preds /= len(cv_models)

sub_cv = pd.DataFrame({id_col: test[id_col], target: cv_test_preds})
save_submission(sub_cv, run_name="felipe_xgb_optuna_cv_only")
print("Saved: felipe_xgb_optuna_cv_only")


# ======================================================
# 8) Full-data multi-seed ensemble
# ======================================================
avg_iter = int(np.mean(fold_best_iters))
seed_list = [42, 100, 200, 300, 400]

dall = xgb.DMatrix(X_full_enc, label=y)

full_seed_preds = np.zeros(len(test_full_enc), dtype=float)
for seed in seed_list:
    p = dict(final_params)
    p["seed"] = seed
    m = xgb.train(p, dall, num_boost_round=avg_iter + 1, evals=[], verbose_eval=False)
    full_seed_preds += m.predict(dtest, iteration_range=(0, avg_iter + 1))
full_seed_preds /= len(seed_list)

sub_full = pd.DataFrame({id_col: test[id_col], target: full_seed_preds})
save_submission(sub_full, run_name="felipe_xgb_optuna_full_seed_only")
print("Saved: felipe_xgb_optuna_full_seed_only")


# ======================================================
# 9) Full-style OOF preds for blending
# ======================================================
full_oof = np.zeros(len(X_base), dtype=float)

print("\nGenerating full-style OOF for blend search...")
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_base), 1):
    X_tr_base = X_base.iloc[tr_idx]
    X_val_base = X_base.iloc[val_idx]
    y_tr = y.iloc[tr_idx]

    X_tr, X_val, _ = fold_target_encode(X_tr_base, y_tr, X_val_base, test_base)
    X_tr  = add_farm_deltas(X_tr)
    X_val = add_farm_deltas(X_val)

    X_tr  = pd.get_dummies(X_tr, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)
    X_val = pd.get_dummies(X_val, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)

    dtr = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_val)

    fold_preds = np.zeros(len(val_idx), dtype=float)
    for seed in seed_list:
        p = dict(final_params)
        p["seed"] = seed
        m = xgb.train(p, dtr, num_boost_round=avg_iter + 1, evals=[], verbose_eval=False)
        fold_preds += m.predict(dval, iteration_range=(0, avg_iter + 1))

    full_oof[val_idx] = fold_preds / len(seed_list)


# ======================================================
# 10) Best alpha blend search
# ======================================================
alphas = np.linspace(0, 1, 201)
best_alpha = 0.5
best_rmse = float("inf")

for a in alphas:
    blend_oof = a * cv_oof + (1 - a) * full_oof
    r = float(np.sqrt(mean_squared_error(y, blend_oof)))
    if r < best_rmse:
        best_rmse = r
        best_alpha = a

print(f"\nBest alpha (CV weight): {best_alpha:.3f}")
print(f"Best blended OOF RMSE : {best_rmse:.6f}")

final_preds = best_alpha * cv_test_preds + (1 - best_alpha) * full_seed_preds

sub_best = pd.DataFrame({id_col: test[id_col], target: final_preds})
save_submission(sub_best, run_name=f"felipe_xgb_platinum_blend_{best_alpha:.3f}")
print("Saved best blend submission.")

[I 2025-11-22 23:15:02,518] A new study created in memory with name: no-name-9ba5b878-5ac9-4be3-ad80-338783f20319



Starting Diamond-Tier XGBoost Optuna (500 trials)...


Best trial: 0. Best value: 4.13814:   0%|          | 1/500 [00:17<2:22:18, 17.11s/it]

[I 2025-11-22 23:15:19,629] Trial 0 finished with value: 4.138135529635429 and parameters: {'grow_policy': 'depthwise', 'max_depth': 8, 'max_bin': 512, 'learning_rate': 0.07540655503925266, 'max_leaves': 256, 'min_child_weight': 1.3172009219182441, 'subsample': 0.6056917652957684, 'colsample_bytree': 0.9149063914253635, 'colsample_bylevel': 0.7560134444478626, 'colsample_bynode': 0.9321921750680274, 'gamma': 0.633456182138236, 'reg_alpha': 37.58863945232069, 'reg_lambda': 0.012132172536105143}. Best is trial 0 with value: 4.138135529635429.


Best trial: 1. Best value: 4.11803:   0%|          | 2/500 [00:33<2:20:34, 16.94s/it]

...

[I 2025-11-23 05:33:48,793] Trial 497 finished with value: 4.11575301200317 and parameters: {'grow_policy': 'depthwise', 'max_depth': 5, 'max_bin': 256, 'learning_rate': 0.007328937914610812, 'max_leaves': 152, 'min_child_weight': 2.6404035590940538, 'subsample': 0.5068902029989973, 'colsample_bytree': 0.5318666630749975, 'colsample_bylevel': 0.9014203140776211, 'colsample_bynode': 0.9000776454789677, 'gamma': 0.8127735257227715, 'reg_alpha': 36.06968499316861, 'reg_lambda': 0.338532894675142}. Best is trial 415 with value: 4.115123725059022.


Best trial: 415. Best value: 4.11512: 100%|█████████▉| 499/500 [6:19:19<00:46, 46.33s/it]

[I 2025-11-23 05:34:22,265] Trial 498 finished with value: 4.119948145954942 and parameters: {'grow_policy': 'depthwise', 'max_depth': 8, 'max_bin': 256, 'learning_rate': 0.006873606666245423, 'max_leaves': 163, 'min_child_weight': 2.8523492306231484, 'subsample': 0.5046322608726452, 'colsample_bytree': 0.528736457292345, 'colsample_bylevel': 0.9043712449480428, 'colsample_bynode': 0.9018507829369526, 'gamma': 0.8478872487766725, 'reg_alpha': 37.11546765633884, 'reg_lambda': 0.31435419599687564}. Best is trial 415 with value: 4.115123725059022.


Best trial: 415. Best value: 4.11512: 100%|██████████| 500/500 [6:20:05<00:00, 45.61s/it]


[I 2025-11-23 05:35:08,262] Trial 499 finished with value: 4.115408542681766 and parameters: {'grow_policy': 'depthwise', 'max_depth': 4, 'max_bin': 256, 'learning_rate': 0.006733628836103081, 'max_leaves': 160, 'min_child_weight': 2.9542148435006057, 'subsample': 0.5390583315251041, 'colsample_bytree': 0.5088939646828724, 'colsample_bylevel': 0.861491663400177, 'colsample_bynode': 0.8794476552829498, 'gamma': 0.37562733709624346, 'reg_alpha': 23.16383142385989, 'reg_lambda': 0.20831649274746436}. Best is trial 415 with value: 4.115123725059022.

Best Params: {'grow_policy': 'depthwise', 'max_depth': 4, 'max_bin': 128, 'learning_rate': 0.006807542126783938, 'max_leaves': 171, 'min_child_weight': 2.8803643269910313, 'subsample': 0.5293099455741657, 'colsample_bytree': 0.5275894416747775, 'colsample_bylevel': 0.9198507961630028, 'colsample_bynode': 0.9413057910082173, 'gamma': 0.6316823862950522, 'reg_alpha': 49.95634643015541, 'reg_lambda': 35.11913756388966, 'objective': 'reg:squareder

In [75]:
import os, sys
import importlib
import json

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb      # still imported in case you need it later
import xgboost as xgb       # same
from catboost import CatBoostRegressor

import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.integration import CatBoostPruningCallback


# ======================================================
# 0) Load data
# ======================================================
train = pd.read_csv("../../data/cattle_data_train.csv")
test  = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) Your existing preprocessing pipeline (UNCHANGED)
# ======================================================
def preprocess_pipeline(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()
    
    # -----------------------
    # 1) Drop impossible targets (train only)
    # -----------------------
    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    # -----------------------
    # 2) Basic cleaning
    # -----------------------
    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )
    
    # Fill any remaining numeric NaNs
    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    # -----------------------
    # 3) Date features
    # -----------------------
    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])
    
    # -----------------------
    # 5) Farm clustering (your current version)
    # -----------------------
    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()

        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)

        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    # -----------------------
    # 6) Drop raw IDs
    # -----------------------
    drop_cols = ["Cattle_ID", "Farm_ID"]
    milk_features = milk_features.drop(columns=[c for c in drop_cols if c in milk_features.columns])

    # -----------------------
    # 7) Optional one-hot encoding (we'll keep it OFF for CatBoost)
    # -----------------------
    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y


# ======================================================
# 2) Build X, y, test_df (frozen pipeline)
# ======================================================
X, y = preprocess_pipeline(train, encode_flag=False, target_col=target)
test_df, _ = preprocess_pipeline(test, encode_flag=False, target_col=None)

print("Train shape:", X.shape)
print("Test shape:", test_df.shape)

cat_features = X.select_dtypes(include="object").columns.tolist()
print("CatBoost categorical features:", cat_features)

kf = KFold(n_splits=5, shuffle=True, random_state=42)


# ======================================================
# Fixed params (locked architecture)
# ======================================================
fixed_params = {
    "depth": 6,
    "bootstrap_type": "Bayesian",
    "grow_policy": "SymmetricTree",
    "use_best_model": True,

    # Locked good settings from your logs
    "border_count": 128,
}


# ======================================================
# 3) Optuna hyperparameter search
#     (Bimodal + tighter mines + real pruning)
# ======================================================
def objective(trial):
    # --- Basin choice first to avoid dead-zone combos ---
    basin = trial.suggest_categorical("basin", ["Mine_A", "Mine_B"])

    # --- Shared tuned params (both mines like these ranges) ---
    tuned_params = {
        "rsm": trial.suggest_float("rsm", 0.6, 1.0),
    }

    # --- Mine-specific narrowed ranges (cut tail waste) ---
    if basin == "Mine_A":
        # Slow & Random recipe
        tuned_params.update({
            "learning_rate": trial.suggest_float("learning_rate", 0.0140, 0.0168, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.7, 2.3),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 1.9, 2.6),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 42, 52),
            "random_strength": trial.suggest_float("random_strength", 0.1, 0.6),
        })
    else:
        # Precise & Clean recipe
        tuned_params.update({
            "learning_rate": trial.suggest_float("learning_rate", 0.0215, 0.0255, log=True),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 5.2, 7.0),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.12, 0.35),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 40, 50),
            "random_strength": trial.suggest_float("random_strength", 0.4, 0.9),
        })

    params = {**fixed_params, **tuned_params}
    fold_rmses = []

    for fold_i, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Prune only on Fold 0 so Hyperband gets a clean monotonic signal
        pruning_cb = None
        callbacks = []
        if fold_i == 0:
            pruning_cb = CatBoostPruningCallback(trial, "RMSE")
            callbacks = [pruning_cb]

        model = CatBoostRegressor(
            loss_function="RMSE",
            eval_metric="RMSE",
            n_estimators=3000,
            early_stopping_rounds=50,   # fast kill only for tuning
            random_seed=42,
            thread_count=4,
            verbose=False,
            allow_writing_files=False,
            **params
        )

        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            cat_features=cat_features,
            callbacks=callbacks,
            verbose=False
        )

        # REQUIRED so CatBoost pruning actually triggers
        if pruning_cb is not None:
            pruning_cb.check_pruned()

        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_rmses.append(rmse)

    return float(np.mean(fold_rmses))


# Trials: enough to refine inside two tight basins
N_TRIALS = 60  # bump to 80 if you want a bit more refinement

print("\nStarting Efficient Bimodal Optuna search...")

sampler = TPESampler(multivariate=True, group=True, seed=42)
pruner = HyperbandPruner(
    min_resource=50,    # match CatBoost ES so pruning can kick in early
    max_resource=3000,
    reduction_factor=3
)

study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)

# Warm-start with your known peaks so TPE learns fast
study.enqueue_trial({
    "basin": "Mine_A",
    "rsm": 0.8,
    "learning_rate": 0.0150,
    "l2_leaf_reg": 1.94,
    "bagging_temperature": 2.16,
    "min_data_in_leaf": 44,
    "random_strength": 0.30
})
study.enqueue_trial({
    "basin": "Mine_B",
    "rsm": 0.8,
    "learning_rate": 0.0240,
    "l2_leaf_reg": 6.5,
    "bagging_temperature": 0.17,
    "min_data_in_leaf": 46,
    "random_strength": 0.60
})

study.optimize(objective, n_trials=N_TRIALS, n_jobs=2, show_progress_bar=True)

best_params = study.best_params
best_cv_rmse = study.best_value

# Remove helper key and rebuild final_params
best_params.pop("basin", None)
final_params = {**fixed_params, **best_params}

print("\nOptuna search complete.")
print("Best CV RMSE from Optuna:", best_cv_rmse)
print("Best tuned params:", best_params)
print("Final params used for training:", final_params)

with open("catboost_best_params.json", "w") as f:
    json.dump(final_params, f, indent=2)
print("Saved best params to catboost_best_params.json")


# ======================================================
# 4) Final 5-fold CV ensemble with final_params
# ======================================================
cv_models = []
fold_rmses = []
fold_best_iters = []

print("\nTraining final 5-fold ensemble with final_params...")

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n----- Final CV Fold {fold} -----")
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # NOTE: do NOT pass use_best_model twice (already in final_params)
    model = CatBoostRegressor(
        loss_function="RMSE",
        eval_metric="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,  # slower ES for final training
        random_seed=42,
        thread_count=4,
        verbose=False,
        allow_writing_files=False,
        **final_params
    )

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    preds_val = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds_val))
    best_iter = model.get_best_iteration()

    print(f"Fold {fold} RMSE: {rmse:.4f}, best_iter={best_iter}")

    fold_rmses.append(rmse)
    fold_best_iters.append(best_iter)
    cv_models.append(model)

final_cv_rmse = float(np.mean(fold_rmses))
print("\n===============================")
print(f"Final 5-fold ensemble CV RMSE: {final_cv_rmse:.4f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)

if final_cv_rmse > best_cv_rmse + 0.001:
    print("⚠️ WARNING: Final CV RMSE is worse than Optuna's best!")
    print(f"Optuna best: {best_cv_rmse:.4f}, Final CV: {final_cv_rmse:.4f}")
    print("This might indicate some instability / differences in folds.")
print("===============================")


# ======================================================
# 5) CV-ensemble predictions on test_df (SAFE baseline)
# ======================================================
cv_test_preds = np.zeros(len(test_df), dtype=float)

for model in cv_models:
    cv_test_preds += model.predict(test_df)

cv_test_preds /= len(cv_models)

sub_cv = pd.DataFrame({
    id_col: test[id_col],
    target: cv_test_preds
})

save_submission(sub_cv, run_name="felipe_catboost_optuna_cv_only")
print("Saved CV-only submission: felipe_catboost_optuna_cv_only")


# ======================================================
# 6) Optional: full-data multi-seed ensemble
# ======================================================
avg_best_iter = int(np.mean(fold_best_iters))
print("\nAverage best_iteration across folds:", avg_best_iter)

n_estimators_full = avg_best_iter
print("Using n_estimators for full-data models:", n_estimators_full)

seed_list = [101, 202, 303, 404, 505]
full_seed_models = []
full_seed_preds = np.zeros(len(test_df), dtype=float)

# IMPORTANT: full-data fits have no eval_set, so remove use_best_model
final_params_no_eval = final_params.copy()
final_params_no_eval.pop("use_best_model", None)

for seed in seed_list:
    print(f"\nTraining full-data model with seed={seed}...")
    model = CatBoostRegressor(
        loss_function="RMSE",
        eval_metric="RMSE",
        n_estimators=n_estimators_full,
        random_seed=seed,
        thread_count=4,
        verbose=False,
        allow_writing_files=False,
        **final_params_no_eval
    )

    model.fit(
        X, y,
        cat_features=cat_features,
        verbose=False
    )

    full_seed_models.append(model)
    full_seed_preds += model.predict(test_df)

full_seed_preds /= len(seed_list)

sub_full = pd.DataFrame({
    id_col: test[id_col],
    target: full_seed_preds
})

save_submission(sub_full, run_name="felipe_catboost_optuna_full_seed_only")
print("Saved full-data seed ensemble submission: felipe_catboost_optuna_full_seed_only")


# ======================================================
# 7) Blended submissions (CV + full-data)
# ======================================================
alpha_70 = 0.7
blend_70_30 = alpha_70 * cv_test_preds + (1.0 - alpha_70) * full_seed_preds

sub_blend_70_30 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_70_30
})
save_submission(sub_blend_70_30, run_name="felipe_catboost_optuna_blend_70_30")
print("Saved blend 70/30 submission: felipe_catboost_optuna_blend_70_30")

alpha_50 = 0.5
blend_50_50 = alpha_50 * cv_test_preds + (1.0 - alpha_50) * full_seed_preds

sub_blend_50_50 = pd.DataFrame({
    id_col: test[id_col],
    target: blend_50_50
})
save_submission(sub_blend_50_50, run_name="felipe_catboost_optuna_blend_50_50")
print("Saved blend 50/50 submission: felipe_catboost_optuna_blend_50_50")


# ======================================================
# 8) EXTRA: OOF RMSE comparison of strategies
# ======================================================
print("\n\n=== Running OOF comparison for strategies (offline) ===")

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

cv_only_oof        = np.zeros(len(X), dtype=float)
full_style_oof     = np.zeros(len(X), dtype=float)
blend_70_30_oof    = np.zeros(len(X), dtype=float)
blend_50_50_oof    = np.zeros(len(X), dtype=float)

for fold, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    print(f"\n[OOF Eval] Fold {fold}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    cv_model = cv_models[fold - 1]
    cv_preds_val = cv_model.predict(X_val)

    fold_full_preds = np.zeros(len(val_idx), dtype=float)
    for seed in seed_list:
        model = CatBoostRegressor(
            loss_function="RMSE",
            eval_metric="RMSE",
            n_estimators=n_estimators_full,
            random_seed=seed,
            verbose=False,
            thread_count=4,
            allow_writing_files=False,
            **final_params_no_eval   # no eval_set here either
        )
        model.fit(
            X_train, y_train,
            cat_features=cat_features,
            verbose=False
        )
        fold_full_preds += model.predict(X_val)

    fold_full_preds /= len(seed_list)

    cv_only_oof[val_idx]     = cv_preds_val
    full_style_oof[val_idx]  = fold_full_preds
    blend_70_30_oof[val_idx] = alpha_70 * cv_preds_val + (1.0 - alpha_70) * fold_full_preds
    blend_50_50_oof[val_idx] = alpha_50 * cv_preds_val + (1.0 - alpha_50) * fold_full_preds

rmse_cv_only     = rmse(y, cv_only_oof)
rmse_full_style  = rmse(y, full_style_oof)
rmse_blend_70_30 = rmse(y, blend_70_30_oof)
rmse_blend_50_50 = rmse(y, blend_50_50_oof)

print("\n=== OOF RMSE summary (lower is better) ===")
print(f"CV-only ensemble OOF RMSE:         {rmse_cv_only:.6f}")
print(f"Full-seed-style ensemble OOF RMSE: {rmse_full_style:.6f}")
print(f"Blend 70% CV / 30% full OOF RMSE:  {rmse_blend_70_30:.6f}")
print(f"Blend 50% CV / 50% full OOF RMSE:  {rmse_blend_50_50:.6f}")


# ======================================================
# 9) Find best alpha blend by OOF (NOT hardcoded)
# ======================================================
alphas = np.linspace(0, 1, 201)
best_alpha = None
best_alpha_rmse = float("inf")

for a in alphas:
    blend_oof = a * cv_only_oof + (1 - a) * full_style_oof
    r = rmse(y, blend_oof)
    if r < best_alpha_rmse:
        best_alpha_rmse = r
        best_alpha = a

print("\n=== Best OOF blend search ===")
print(f"Best alpha (CV weight) = {best_alpha:.3f}")
print(f"Best blended OOF RMSE  = {best_alpha_rmse:.6f}")

best_blend_test = best_alpha * cv_test_preds + (1 - best_alpha) * full_seed_preds

sub_best_blend = pd.DataFrame({
    id_col: test[id_col],
    target: best_blend_test
})

save_submission(sub_best_blend, run_name=f"felipe_catboost_optuna_best_blend_{best_alpha:.3f}")
print("Saved best-alpha blend submission.")

ModuleNotFoundError: 
Could not find `optuna-integration` for `catboost`.
Please run `pip install optuna-integration[catboost]`.

# ATTEMPTING TO STACK

In [ ]:
import os, sys, importlib, json
import numpy as np
import pandas as pd
import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge

# submission utils
sys.path.append(os.path.abspath(".."))
import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission


# ======================================================
# 0) Load data
# ======================================================
train = pd.read_csv("../../data/cattle_data_train.csv")
test  = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) CATBOOST preprocessing (your pipeline)
# ======================================================
def preprocess_pipeline_cb(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()

    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )

    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])

    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    milk_features = milk_features.drop(columns=["Cattle_ID", "Farm_ID"], errors="ignore")

    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y


# ======================================================
# 2) XGBOOST preprocessing (your pipeline)
# ======================================================
def preprocess_pipeline_xgb(df, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()

    if target_col is not None and target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0].copy()
        y = milk_features[target_col].copy()
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].astype(str).str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )

    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    if len(numeric_cols) > 0:
        milk_features[numeric_cols] = milk_features[numeric_cols].fillna(
            milk_features[numeric_cols].median()
        )

    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"], errors="coerce")
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])

    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]
        if len(farm_numeric_cols) > 0:
            farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()
            scaler = StandardScaler()
            farm_scaled = scaler.fit_transform(farm_features)
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
            farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)
            milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])
        else:
            milk_features["Farm_Cluster"] = 0

    vax_cols = [c for c in milk_features.columns if c.endswith("_Vaccine")]
    if len(vax_cols) > 0:
        milk_features["Vax_Sum"] = milk_features[vax_cols].sum(axis=1)

    if "Parity" in milk_features.columns:
        milk_features["First_Calf"] = (milk_features["Parity"] == 1).astype(int)
        milk_features["Prime_Cow"] = ((milk_features["Parity"] >= 2) & (milk_features["Parity"] <= 4)).astype(int)
        milk_features["Old_Cow"]   = (milk_features["Parity"] > 4).astype(int)

    milk_features = milk_features.drop(columns=["Cattle_ID"], errors="ignore")

    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y


def fold_target_encode(train_df, y_train, val_df, test_df,
                       col="Farm_ID", new_name="Farm_Performance"):
    train_df = train_df.copy()
    val_df   = val_df.copy()
    test_df  = test_df.copy()

    global_avg = float(y_train.mean())
    means = y_train.groupby(train_df[col]).mean()

    train_df[new_name] = train_df[col].map(means).fillna(global_avg)
    val_df[new_name]   = val_df[col].map(means).fillna(global_avg)
    test_df[new_name]  = test_df[col].map(means).fillna(global_avg)

    train_df = train_df.drop(columns=[col], errors="ignore")
    val_df   = val_df.drop(columns=[col], errors="ignore")
    test_df  = test_df.drop(columns=[col], errors="ignore")

    return train_df, val_df, test_df


def add_farm_deltas(df_):
    df_ = df_.copy()
    if "Previous_Week_Avg_Yield" in df_.columns and "Farm_Performance" in df_.columns:
        df_["Prev_vs_Farm"]   = df_["Previous_Week_Avg_Yield"] - df_["Farm_Performance"]
        df_["Prev_over_Farm"] = df_["Previous_Week_Avg_Yield"] / (df_["Farm_Performance"] + 1e-6)
    return df_


# ======================================================
# 3) Build base datasets
# ======================================================
# CatBoost base
X_cb, y_cb = preprocess_pipeline_cb(train, encode_flag=False, target_col=target)
test_cb, _ = preprocess_pipeline_cb(test, encode_flag=False, target_col=None)
cat_features = X_cb.select_dtypes(include="object").columns.tolist()

# XGB base
X_base, y = preprocess_pipeline_xgb(train, target_col=target)
test_base, _ = preprocess_pipeline_xgb(test, target_col=None)

X_base = X_base.reset_index(drop=True)
y = y.reset_index(drop=True)
test_base = test_base.reset_index(drop=True)

# Same KFold splits for alignment
kf = KFold(n_splits=5, shuffle=True, random_state=42)


# ======================================================
# 4) Best params (from your logs)
# ======================================================
catboost_best = {
    "depth": 6,
    "learning_rate": 0.022872270426979868,
    "l2_leaf_reg": 4.014881862534349,
    "subsample": 0.847236734294946,          # harmless if ignored under Bayesian
    "random_strength": 0.731107887920747,
    "bagging_temperature": 0.4635035306948895
}

xgb_best = {
    "grow_policy": "depthwise",
    "max_depth": 4,
    "max_bin": 128,
    "learning_rate": 0.006807542126783938,
    "max_leaves": 171,
    "min_child_weight": 2.8803643269910313,
    "subsample": 0.5293099455741657,
    "colsample_bytree": 0.5275894416747775,
    "colsample_bylevel": 0.9198507961630028,
    "colsample_bynode": 0.9413057910082173,
    "gamma": 0.6316823862950522,
    "reg_alpha": 49.95634643015541,
    "reg_lambda": 35.11913756388966,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "seed": 42,
    "nthread": 4
}


# ======================================================
# 5) GLOBAL feature template for XGB (prevents mismatch)
# ======================================================
X_full_enc, _, test_full_enc = fold_target_encode(X_base, y, X_base.iloc[:1], test_base)
X_full_enc = add_farm_deltas(X_full_enc)
test_full_enc = add_farm_deltas(test_full_enc)

X_full_enc = pd.get_dummies(X_full_enc, drop_first=False)
test_full_enc = pd.get_dummies(test_full_enc, drop_first=False)
X_full_enc, test_full_enc = X_full_enc.align(test_full_enc, join="left", axis=1, fill_value=0)

GLOBAL_COLS = X_full_enc.columns.tolist()
dtest_global = xgb.DMatrix(test_full_enc)


# ======================================================
# 6) Rebuild CatBoost OOF + Test preds
# ======================================================
print("\nRebuilding CatBoost OOF + test preds...")
cb_oof = np.zeros(len(X_cb))
cb_models = []
cb_best_iters = []
cb_fold_rmses = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_cb), 1):
    X_tr, X_val = X_cb.iloc[tr_idx], X_cb.iloc[val_idx]
    y_tr, y_val = y_cb.iloc[tr_idx], y_cb.iloc[val_idx]

    cb = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,
        random_seed=42,
        thread_count=4,
        verbose=False,
        use_best_model=True,
        **catboost_best
    )
    cb.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    best_iter = cb.get_best_iteration()
    preds = cb.predict(X_val)
    rmse = float(np.sqrt(mean_squared_error(y_val, preds)))

    print(f"CB Fold {fold} RMSE: {rmse:.6f}, best_iter={best_iter}")

    cb_oof[val_idx] = preds
    cb_models.append(cb)
    cb_best_iters.append(best_iter)
    cb_fold_rmses.append(rmse)

cb_cv_rmse = float(np.sqrt(mean_squared_error(y_cb, cb_oof)))
print(f"CatBoost OOF RMSE: {cb_cv_rmse:.6f}")

cb_test_preds = np.mean([m.predict(test_cb) for m in cb_models], axis=0)


# ======================================================
# 7) Rebuild XGBoost OOF + Test preds
# ======================================================
print("\nRebuilding XGBoost OOF + test preds...")
xgb_oof = np.zeros(len(X_base))
xgb_models = []
xgb_best_iters = []
xgb_fold_rmses = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_base), 1):
    X_tr_base = X_base.iloc[tr_idx]
    X_val_base = X_base.iloc[val_idx]
    y_tr = y.iloc[tr_idx]
    y_val = y.iloc[val_idx]

    # fold-safe TE
    X_tr, X_val, _ = fold_target_encode(X_tr_base, y_tr, X_val_base, test_base)
    X_tr = add_farm_deltas(X_tr)
    X_val = add_farm_deltas(X_val)

    # OHE + align to GLOBAL_COLS
    X_tr = pd.get_dummies(X_tr, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)
    X_val = pd.get_dummies(X_val, drop_first=False).reindex(columns=GLOBAL_COLS, fill_value=0)

    dtr = xgb.DMatrix(X_tr, label=y_tr)
    dval = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params=xgb_best,
        dtrain=dtr,
        num_boost_round=6000,
        evals=[(dval, "val")],
        early_stopping_rounds=120,
        verbose_eval=False
    )

    best_iter = model.best_iteration if hasattr(model, "best_iteration") else 5999
    preds = model.predict(dval, iteration_range=(0, best_iter + 1))
    rmse = float(np.sqrt(mean_squared_error(y_val, preds)))

    print(f"XGB Fold {fold} RMSE: {rmse:.6f}, best_iter={best_iter}")

    xgb_oof[val_idx] = preds
    xgb_models.append(model)
    xgb_best_iters.append(best_iter)
    xgb_fold_rmses.append(rmse)

xgb_cv_rmse = float(np.sqrt(mean_squared_error(y, xgb_oof)))
print(f"XGBoost OOF RMSE: {xgb_cv_rmse:.6f}")

# test preds = average of fold models using their own best_iter
xgb_test_preds = np.zeros(len(test_full_enc))
for m, bi in zip(xgb_models, xgb_best_iters):
    xgb_test_preds += m.predict(dtest_global, iteration_range=(0, bi + 1))
xgb_test_preds /= len(xgb_models)


# ======================================================
# 8) Linear alpha stacking (OOF search)
# ======================================================
def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

print("\nAlpha search between CatBoost and XGBoost...")
alphas = np.linspace(0, 1, 401)  # 0.0025 steps
best_a, best_r = None, float("inf")

for a in alphas:
    blend_oof = a * cb_oof + (1 - a) * xgb_oof
    r = rmse(y_cb, blend_oof)
    if r < best_r:
        best_r, best_a = r, a

print(f"Best alpha (CB weight) = {best_a:.4f}")
print(f"Best 2-model blended OOF RMSE = {best_r:.6f}")

stack_test_alpha = best_a * cb_test_preds + (1 - best_a) * xgb_test_preds
sub_alpha = pd.DataFrame({id_col: test[id_col], target: stack_test_alpha})
# save_submission(sub_alpha, run_name=f"felipe_stack_cb_xgb_alpha_{best_a:.4f}")
print("Saved alpha-stacked submission.")


# ======================================================
# 9) Ridge meta-stacking (still just 2-model linear stack)
# ======================================================
print("\nRidge meta-stacking on OOF preds...")
Z = np.vstack([cb_oof, xgb_oof]).T  # shape (n, 2)
ridge_alphas = np.logspace(-4, 3, 50)

best_ra, best_rr = None, float("inf")
best_ridge = None

# simple CV over ridge strength on OOF (cheap)
for ra in ridge_alphas:
    # fit ridge on full OOF, eval via same OOF (OK since OOF already leak-safe)
    model = Ridge(alpha=ra, fit_intercept=True)
    model.fit(Z, y_cb)
    preds = model.predict(Z)
    r = rmse(y_cb, preds)

    if r < best_rr:
        best_rr, best_ra, best_ridge = r, ra, model

print(f"Best Ridge alpha = {best_ra:.6f}")
print(f"Best Ridge-stacked OOF RMSE = {best_rr:.6f}")
print("Ridge weights (CB, XGB) =", best_ridge.coef_, "intercept =", best_ridge.intercept_)

Z_test = np.vstack([cb_test_preds, xgb_test_preds]).T
stack_test_ridge = best_ridge.predict(Z_test)

sub_ridge = pd.DataFrame({id_col: test[id_col], target: stack_test_ridge})
save_submission(sub_ridge, run_name=f"felipe_stack_cb_xgb_ridge_{best_ra:.6f}")
print("Saved ridge-stacked submission.")


print("\nDONE. Compare alpha-stack vs ridge-stack RMSE above.")


Rebuilding CatBoost OOF + test preds...
CB Fold 1 RMSE: 4.106957, best_iter=1157
CB Fold 2 RMSE: 4.100151, best_iter=1127
CB Fold 3 RMSE: 4.120099, best_iter=1093
CB Fold 4 RMSE: 4.106328, best_iter=1094
CB Fold 5 RMSE: 4.098421, best_iter=999
CatBoost OOF RMSE: 4.106398

Rebuilding XGBoost OOF + test preds...
XGB Fold 1 RMSE: 4.116921, best_iter=2787
XGB Fold 2 RMSE: 4.108427, best_iter=3056
XGB Fold 3 RMSE: 4.127091, best_iter=3277
XGB Fold 4 RMSE: 4.116688, best_iter=2884
XGB Fold 5 RMSE: 4.106360, best_iter=2886
XGBoost OOF RMSE: 4.115104

Alpha search between CatBoost and XGBoost...
Best alpha (CB weight) = 0.8675
Best 2-model blended OOF RMSE = 4.106189
Saved alpha-stacked submission.

Ridge meta-stacking on OOF preds...
Best Ridge alpha = 0.000100
Best Ridge-stacked OOF RMSE = 4.105955
Ridge weights (CB, XGB) = [0.87095368 0.14202604] intercept = -0.20003385945580554
Saved submission -> /Users/felipebenitez/ML-Project/submissions/felipe_stack_cb_xgb_ridge_0.000100__2025-11-23__

# trying to get better blend for catboost only

In [86]:
# ======================================================
# CatBoost-only:
#   - CV-only ensemble (exactly your training)
#   - Full-seed ensemble (multi-seed on full data)
#   - Full-style OOF (per-fold multi-seed, no leakage)
#   - Best alpha blend search (CV vs Full)
#   - Best ridge stack (CV vs Full)
# ======================================================

import os, sys, importlib
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge

# submission utils
sys.path.append(os.path.abspath(".."))
import submission_utils
importlib.reload(submission_utils)
from submission_utils import save_submission


# ======================================================
# 0) Load data
# ======================================================
train = pd.read_csv("../../data/cattle_data_train.csv")
test  = pd.read_csv("../../data/cattle_data_test.csv")

target = "Milk_Yield_L"
id_col = "Cattle_ID"


# ======================================================
# 1) CATBOOST preprocessing (YOUR pipeline, UNCHANGED)
# ======================================================
def preprocess_pipeline_cb(df, encode_flag=True, target_col="Milk_Yield_L", n_clusters=10):
    milk_features = df.copy()

    if target_col in milk_features.columns:
        milk_features = milk_features[milk_features[target_col] >= 0]
        y = milk_features[target_col]
        milk_features = milk_features.drop(columns=[target_col])
    else:
        y = None

    if "Breed" in milk_features.columns:
        milk_features["Breed"] = milk_features["Breed"].str.strip()
        milk_features["Breed"] = milk_features["Breed"].replace({"Holstien": "Holstein"})

    if "Housing_Score" in milk_features.columns:
        milk_features["Housing_Score"] = milk_features["Housing_Score"].fillna(
            milk_features["Housing_Score"].median()
        )

    if "Feed_Quantity_kg" in milk_features.columns and "Feed_Type" in milk_features.columns:
        milk_features["Feed_Quantity_kg"] = milk_features.groupby("Feed_Type")["Feed_Quantity_kg"].transform(
            lambda x: x.fillna(x.median())
        )

    numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
    milk_features[numeric_cols] = milk_features[numeric_cols].fillna(milk_features[numeric_cols].median())

    if "Date" in milk_features.columns:
        milk_features["Date"] = pd.to_datetime(milk_features["Date"])
        milk_features["year"] = milk_features["Date"].dt.year
        milk_features["month"] = milk_features["Date"].dt.month
        milk_features["day"] = milk_features["Date"].dt.day
        milk_features["dayofweek"] = milk_features["Date"].dt.dayofweek
        milk_features["weekofyear"] = milk_features["Date"].dt.isocalendar().week.astype(int)
        milk_features["quarter"] = milk_features["Date"].dt.quarter
        milk_features["is_weekend"] = milk_features["dayofweek"].isin([5, 6]).astype(int)
        milk_features["date_ordinal"] = milk_features["Date"].map(pd.Timestamp.toordinal)
        milk_features = milk_features.drop(columns=["Date"])

    if "Farm_ID" in milk_features.columns:
        farm_numeric_cols = milk_features.select_dtypes(include="number").columns.tolist()
        farm_numeric_cols = [c for c in farm_numeric_cols if c not in ["Cattle_ID"]]

        farm_features = milk_features.groupby("Farm_ID")[farm_numeric_cols].mean()
        scaler = StandardScaler()
        farm_scaled = scaler.fit_transform(farm_features)
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        farm_features["Cluster"] = kmeans.fit_predict(farm_scaled)
        milk_features["Farm_Cluster"] = milk_features["Farm_ID"].map(farm_features["Cluster"])

    milk_features = milk_features.drop(columns=["Cattle_ID", "Farm_ID"], errors="ignore")

    if encode_flag:
        cat_cols = milk_features.select_dtypes(include="object").columns.tolist()
        milk_features = pd.get_dummies(milk_features, columns=cat_cols, drop_first=False)

    if "Farm_Cluster" not in milk_features.columns:
        milk_features["Farm_Cluster"] = 0

    return milk_features.copy(), y


# ======================================================
# 2) Build base datasets (CatBoost only)
# ======================================================
X_cb, y_cb = preprocess_pipeline_cb(train, encode_flag=False, target_col=target)
test_cb, _ = preprocess_pipeline_cb(test, encode_flag=False, target_col=None)
cat_features = X_cb.select_dtypes(include="object").columns.tolist()

kf = KFold(n_splits=5, shuffle=True, random_state=42)

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))


# ======================================================
# 3) Best params (FROM YOUR LOGS, UNCHANGED)
# ======================================================
catboost_best = {
    "depth": 6,
    "learning_rate": 0.022872270426979868,
    "l2_leaf_reg": 4.014881862534349,
    "subsample": 0.847236734294946,
    "random_strength": 0.731107887920747,
    "bagging_temperature": 0.4635035306948895
}

SEED_LIST = [101, 202, 303, 404, 505]


# ======================================================
# 4) CV-only OOF + test preds (YOUR exact training)
# ======================================================
print("\nRebuilding CatBoost CV-only OOF + test preds...")
cv_only_oof = np.zeros(len(X_cb))
cv_models = []
fold_best_iters = []
fold_rmses = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_cb), 1):
    X_tr, X_val = X_cb.iloc[tr_idx], X_cb.iloc[val_idx]
    y_tr, y_val = y_cb.iloc[tr_idx], y_cb.iloc[val_idx]

    cb = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=3000,
        early_stopping_rounds=100,
        random_seed=42,
        thread_count=4,
        verbose=False,
        use_best_model=True,
        **catboost_best
    )
    cb.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
        verbose=False
    )

    best_iter = cb.get_best_iteration()
    preds_val = cb.predict(X_val)
    fold_rmse = rmse(y_val, preds_val)

    print(f"CB Fold {fold} RMSE: {fold_rmse:.6f}, best_iter={best_iter}")

    cv_only_oof[val_idx] = preds_val
    cv_models.append(cb)
    fold_best_iters.append(best_iter)
    fold_rmses.append(fold_rmse)

cv_only_rmse = rmse(y_cb, cv_only_oof)
avg_best_iter = int(np.mean(fold_best_iters))

print("\n===============================")
print(f"CV-only ensemble OOF RMSE: {cv_only_rmse:.6f}")
print("Fold RMSEs:", fold_rmses)
print("Best iters:", fold_best_iters)
print("Average best_iteration across folds:", avg_best_iter)
print("===============================\n")

# CV-only test preds (average of fold models, like you do)
cv_test_preds = np.mean([m.predict(test_cb) for m in cv_models], axis=0)


# ======================================================
# 5) Full-style OOF + full-seed test preds
#     (matches your earlier CatBoost-only blend logic)
# ======================================================
print("\nRebuilding Full-style OOF + full-seed test preds...")
full_style_oof = np.zeros(len(X_cb))
full_seed_test_preds = np.zeros(len(test_cb))

# Full-style OOF: per fold, train multi-seed on fold-train only
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_cb), 1):
    X_tr, X_val = X_cb.iloc[tr_idx], X_cb.iloc[val_idx]
    y_tr = y_cb.iloc[tr_idx]

    fold_full_preds = np.zeros(len(val_idx), dtype=float)

    for seed in SEED_LIST:
        m = CatBoostRegressor(
            loss_function="RMSE",
            n_estimators=avg_best_iter,
            random_seed=seed,
            thread_count=4,
            verbose=False,
            **catboost_best
        )
        m.fit(X_tr, y_tr, cat_features=cat_features, verbose=False)
        fold_full_preds += m.predict(X_val)

    fold_full_preds /= len(SEED_LIST)
    full_style_oof[val_idx] = fold_full_preds

full_style_rmse = rmse(y_cb, full_style_oof)
print(f"Full-style ensemble OOF RMSE: {full_style_rmse:.6f}")

# Full-seed test preds: multi-seed on full data
for seed in SEED_LIST:
    m = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=avg_best_iter,
        random_seed=seed,
        thread_count=4,
        verbose=False,
        **catboost_best
    )
    m.fit(X_cb, y_cb, cat_features=cat_features, verbose=False)
    full_seed_test_preds += m.predict(test_cb)

full_seed_test_preds /= len(SEED_LIST)


# ======================================================
# 6) Best alpha blend search (CV-only vs Full-style)
# ======================================================
print("\nAlpha search between CV-only and Full-style...")
alphas = np.linspace(0, 1, 201)
best_alpha, best_alpha_rmse = None, float("inf")

for a in alphas:
    blend_oof = a * cv_only_oof + (1 - a) * full_style_oof
    r = rmse(y_cb, blend_oof)
    if r < best_alpha_rmse:
        best_alpha_rmse = r
        best_alpha = a

print("\n=== Best alpha blend (CV vs Full) ===")
print(f"Best alpha (CV weight): {best_alpha:.4f}")
print(f"Best blended OOF RMSE : {best_alpha_rmse:.6f}")

alpha_test_preds = best_alpha * cv_test_preds + (1 - best_alpha) * full_seed_test_preds


# ======================================================
# 7) Ridge meta-stacking (CV-only vs Full-style)
# ======================================================
print("\nRidge meta-stacking on (CV-only, Full-style)...")
meta_X = np.vstack([cv_only_oof, full_style_oof]).T
meta_test = np.vstack([cv_test_preds, full_seed_test_preds]).T

ridge_alphas = np.logspace(-4, 2, 60)
best_ra, best_rr, best_ridge = None, float("inf"), None

for ra in ridge_alphas:
    ridge = Ridge(alpha=ra, fit_intercept=True, random_state=42)
    ridge.fit(meta_X, y_cb)
    preds_oof = ridge.predict(meta_X)
    r = rmse(y_cb, preds_oof)
    if r < best_rr:
        best_rr, best_ra, best_ridge = r, ra, ridge

print("\n=== Best Ridge stack (CV vs Full) ===")
print(f"Best Ridge alpha: {best_ra:.6f}")
print(f"Best Ridge-stacked OOF RMSE: {best_rr:.6f}")
print("Ridge weights (CV, Full):", best_ridge.coef_, "intercept:", best_ridge.intercept_)

ridge_test_preds = best_ridge.predict(meta_test)


# ======================================================
# 8) Save submissions
# ======================================================
sub_cv = pd.DataFrame({id_col: test[id_col], target: cv_test_preds})
save_submission(sub_cv, run_name="felipe_catboost_cv_only_bestparams")
print("Saved CV-only submission.")

sub_full = pd.DataFrame({id_col: test[id_col], target: full_seed_test_preds})
save_submission(sub_full, run_name="felipe_catboost_full_seed_only_bestparams")
print("Saved full-seed-only submission.")

sub_alpha = pd.DataFrame({id_col: test[id_col], target: alpha_test_preds})
save_submission(sub_alpha, run_name=f"felipe_catboost_best_alpha_blend_{best_alpha:.4f}")
print("Saved best-alpha blend submission.")

sub_ridge = pd.DataFrame({id_col: test[id_col], target: ridge_test_preds})
save_submission(sub_ridge, run_name=f"felipe_catboost_ridge_stack_alpha_{best_ra:.6f}")
print("Saved ridge-stacked submission.")

print("\nDONE.")


Rebuilding CatBoost CV-only OOF + test preds...
CB Fold 1 RMSE: 4.106957, best_iter=1157
CB Fold 2 RMSE: 4.100151, best_iter=1127
CB Fold 3 RMSE: 4.120099, best_iter=1093
CB Fold 4 RMSE: 4.106328, best_iter=1094
CB Fold 5 RMSE: 4.098421, best_iter=999

CV-only ensemble OOF RMSE: 4.106398
Fold RMSEs: [4.106956550228758, 4.100151472353626, 4.120099489769896, 4.106327628323443, 4.0984207308223395]
Best iters: [1157, 1127, 1093, 1094, 999]
Average best_iteration across folds: 1094


Rebuilding Full-style OOF + full-seed test preds...
Full-style ensemble OOF RMSE: 4.106304

Alpha search between CV-only and Full-style...

=== Best alpha blend (CV vs Full) ===
Best alpha (CV weight): 0.4400
Best blended OOF RMSE : 4.106145

Ridge meta-stacking on (CV-only, Full-style)...

=== Best Ridge stack (CV vs Full) ===
Best Ridge alpha: 0.000100
Best Ridge-stacked OOF RMSE: 4.105931
Ridge weights (CV, Full): [0.44389453 0.56851011] intercept: -0.1922872770191031
Saved submission -> /Users/felipebenite